In [127]:
import pdfplumber
import pandas as pd
import os
from pprint import pprint
from tabulate import tabulate
new_directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/matched_pdfs_phase_i/Q1030/"
os.chdir(new_directory)


In [186]:
import pdfplumber
import pandas as pd
from tabulate import tabulate
import os

new_directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/matched_pdfs_phase_i/Q1030"
os.chdir(new_directory)

# Initialize an empty DataFrame to store all tables
core = pd.DataFrame()
table_in_progress = pd.DataFrame() 
table_started = False
headers = None  # To store headers of the table that starts on the first page

# Open the PDF file
with pdfplumber.open("C7PhI - Appendix A - Q1030   South Lake Solar.pdf") as pdf:
    for page in pdf.pages:
        # Extract all tables from the page
        tables = page.extract_tables()  # Use extract_tables to get all tables directly

        # Iterate over each table
        for table_index, tab in enumerate(tables):
            # If this is the first part of the table
            if not table_started:
                table_started = True
                headers = tab[0]  # First row is the headers
                print(f"Table {table_index + 1} headers: {headers}")
                # Convert the rest of the table to DataFrame
                table_in_progress = pd.DataFrame(tab[1:], columns=headers)
            else:
                # If it's the continuation of a previous table, append without treating the first row as headers
                continuation_data = pd.DataFrame(tab, columns=headers)  # Use the same headers as before
                table_in_progress = pd.concat([table_in_progress, continuation_data], ignore_index=True)

        # If there were tables on this page, continue accumulating
        # If there were no tables, reset and finalize the current "table in progress"
        if not tables:
            core = pd.concat([core, table_in_progress], ignore_index=True, sort=False)
            table_started = False  # Reset for the next table
            table_in_progress = pd.DataFrame()  # Clear the in-progress table

# After processing all pages, finalize any remaining table in progress
if not table_in_progress.empty:
    core = pd.concat([core, table_in_progress], ignore_index=True, sort=False)

# Removing any rows that contain the header "Type of Upgrade"
core = core[~core.apply(lambda row: row.str.contains("Type of Upgrade", case=False, na=False).any(), axis=1)]

# Print the final combined DataFrame
print(tabulate(core, headers='keys', tablefmt='fancy_grid', showindex=False))


Table 1 headers: ['Project Parameters', 'Project Specific Data']
Table 1 headers: ['Type of Upgrade', 'Upgrade', 'Description', 'Cost\nAllocation\nFactor', 'Estimated\nCost x\n1,000', 'Escalated\nCosts x 1,000', 'Estimated\nTime\n(Months) to\nConstruct\n(Note 1)']
╒════════════════════════════════════╤════════════════════════════════════════════╤═══════════════════╤═══════════════════════════╤═══════════════════════════════════╤══════════════╤═════════════╤═════════════════╤═══════════════╕
│ Project Parameters                 │ Project Specific Data                      │ Type of Upgrade   │ Upgrade                   │ Description                       │ Cost         │ Estimated   │ Escalated       │ Estimated     │
│                                    │                                            │                   │                           │                                   │ Allocation   │ Cost x      │ Costs x 1,000   │ Time          │
│                                    │    

# This is the beauty

In [215]:


import os
import pdfplumber
import pandas as pd
import re

new_directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/matched_pdfs_phase_i/Q1030"
os.chdir(new_directory)

# Initialize an empty DataFrame to store all tables
core = pd.DataFrame()
table_in_progress = pd.DataFrame() 
table_started = False
headers = None  # To store headers of the table that starts on the first page
expected_columns = None  # To track the number of columns expected for a continuation
last_page_number = -1  # Track the last page number for duplicate check
table_continues = False

def merge_headers(existing_df, new_df):
    """Merge columns from new table into the existing DataFrame, filling missing values with NaN."""
    return pd.concat([existing_df, new_df], ignore_index=True, sort=False)


def ensure_unique_headers(headers):
    """Ensure column headers are unique by appending a suffix for duplicates."""
    seen = set()
    new_headers = []
    for col in headers:
        if col in seen:
            count = 1
            new_col = f"{col}_{count}"
            while new_col in seen:
                count += 1
                new_col = f"{col}_{count}"
            new_headers.append(new_col)
            seen.add(new_col)
        else:
            new_headers.append(col)
            seen.add(col)
    return new_headers

# Open the PDF file
with pdfplumber.open("C7PhI - Appendix A - Q1030   South Lake Solar.pdf") as pdf:
    
    for page_number, page in enumerate(pdf.pages):
        text = page.extract_text()
        if re.search(r"Table 7-", text):
            # Extract all tables from the page
            tables = page.find_tables(table_settings={
                "horizontal_strategy": "lines",
                "vertical_strategy": "lines",
            })

            # If no tables found on the page, continue to the next page
            if not tables:
                print(f"No tables found on page {page_number + 1}")
                continue

            # Iterate over each table on the page
            for table_index, table in enumerate(tables):
                tab = table.extract()

                # If the table is empty, skip it
                if len(tab) == 0:
                    continue

                # Extract headers and rows
                current_headers = tab[0]  # First row as headers
                data_rows = tab[1:]  # Data rows
                # Ensure unique column names
                current_headers = ensure_unique_headers(current_headers)

                # If the table has the same number of columns, consider it a continuation
                if headers and len(current_headers) == expected_columns:
                    print(f"Detected continuation on page {page_number + 1}, table {table_index + 1}")
                    continuation_data = pd.DataFrame(data_rows, columns=headers)
                    table_in_progress = pd.concat([table_in_progress, continuation_data], ignore_index=True)
                    table_continues = False
                else:
                    # If a new table starts, append the old table in progress
                    if not table_in_progress.empty and table_continues:
                        print(f"Appending previous table to core from page {page_number + 1}")
                        core = pd.concat([core, table_in_progress], ignore_index=True)
                        table_in_progress = pd.DataFrame()  # Reset the in-progress table

                    # Start a new table
                    headers = current_headers
                    expected_columns = len(headers)
                     
                    print(f"New table detected on page {page_number + 1}, table {table_index + 1} with {expected_columns} columns")
                    table_in_progress = pd.DataFrame(data_rows, columns=headers)
                    table_continues = True

                # Remove duplicate headers inside the data rows
                if not table_in_progress.empty and table_in_progress.iloc[0].equals(pd.Series(headers)):
                    print("Duplicate header row found, removing...")
                    table_in_progress = table_in_progress.drop(0)


        # Append any remaining table in progress to core after the last page
        if not table_in_progress.empty:
            print(f"Appending final table in progress to core")
            core = pd.concat([core, table_in_progress], ignore_index=True)

# Ensure the final DataFrame has all the columns detected (dynamically add new columns)
valid_columns = [col for col in core.columns.unique() if col is not None]  # Remove any NoneType columns
core = core.reindex(columns=valid_columns)
# Print the final combined DataFrame
print(tabulate(core, headers='keys', tablefmt='fancy_grid', showindex=False))

# If you want to export this to a CSV file, uncomment the following line:
# core.to_csv('final_table_data.csv', index=False)

# 

New table detected on page 13, table 1 with 7 columns
Appending final table in progress to core
Detected continuation on page 14, table 1
Appending final table in progress to core
Detected continuation on page 15, table 1
Detected continuation on page 15, table 2
Appending final table in progress to core
Detected continuation on page 16, table 1
Detected continuation on page 16, table 2
Appending final table in progress to core
Appending final table in progress to core
Appending final table in progress to core
╒═══════════════════╤═══════════════════════════╤═══════════════════════════════════╤══════════════╤═════════════╤═════════════════╤═══════════════╕
│ Type of Upgrade   │ Upgrade                   │ Description                       │ Cost         │ Estimated   │ Escalated       │ Estimated     │
│                   │                           │                                   │ Allocation   │ Cost x      │ Costs x 1,000   │ Time          │
│                   │                

In [255]:
#This works the best, gets us what we want

import os
import pdfplumber
import pandas as pd
new_directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/matched_pdfs_phase_i/"
os.chdir(new_directory)

# Initialize an empty DataFrame to store all tables
core = pd.DataFrame()
table_in_progress = pd.DataFrame()
headers = None  # To store headers of the table that starts on the first page
expected_columns = None  # To track the number of columns expected for a continuation

def merge_headers(existing_df, new_df):
    """Merge columns from new table into the existing DataFrame, filling missing values with NaN."""
    return pd.concat([existing_df, new_df], ignore_index=True, sort=False)


# Function to clean up and standardize column headers, including removing brackets and their contents
def clean_column_headers(headers):
    cleaned_headers = []
    for header in headers:
        header = header.lower()  # Convert to lowercase
        header = re.sub(r'\s+', ' ', header)  # Replace multiple spaces with a single space
        header = re.sub(r'\(.*?\)', '', header)  # Remove anything in parentheses, including brackets
        header = re.sub(r'[^a-zA-Z0-9\s]', '', header)  # Remove special characters like $ and commas
        header = header.strip()  # Remove leading/trailing spaces
        cleaned_headers.append(header)
    return cleaned_headers


# Open the PDF file
with pdfplumber.open("Q1040/QC7PhI_Q1040_Arlington Valley Solar Energy 2_Appendix A_12-17-2014.pdf") as pdf:
#with pdfplumber.open("Q1030/C7PhI - Appendix A - Q1030   South Lake Solar.pdf") as pdf:    
    for page_number, page in enumerate(pdf.pages):
         
        text = page.extract_text()
        if re.search(r"Table 7", text):
            # Extract all tables from the page
            tables = page.find_tables(table_settings={
            "horizontal_strategy": "lines",
            "vertical_strategy": "lines",
            })

            # If no tables found on the page, continue to the next page
            if not tables:
                print(f"No tables found on page {page_number + 1}")
                continue

            # Iterate over each table on the page
            for table_index, table in enumerate(tables):
                tab = table.extract()

                # If the table is empty, skip it
                if len(tab) == 0:
                    continue

                # Extract headers and rows
                current_headers = tab[0]  # First row as headers
                cleaned_headers = clean_column_headers(current_headers)
                data_rows = tab[1:]  # Data rows

                # If the table has the same number of columns, consider it a continuation
                if headers and len(current_headers) == expected_columns:
                    print(f"Detected continuation on page {page_number + 1}, table {table_index + 1}")
                    continuation_data = pd.DataFrame(tab, columns=headers)
                    table_in_progress = pd.concat([table_in_progress, continuation_data], ignore_index=True)

                    
                else:
                    # If a new table starts, append the old table in progress
                    if not table_in_progress.empty:
                        print(f"Appending previous table to core from page {page_number + 1}")
                        core = core.join([table_in_progress], how='outer')
                        table_in_progress = pd.DataFrame()  # Reset the in-progress table

                    # Start a new table
                    headers = cleaned_headers
                    expected_columns = len(headers)
                    print(f"New table detected on page {page_number + 1}, table {table_index + 1} with {expected_columns} columns")
                    table_in_progress = pd.DataFrame(data_rows, columns=headers)

                 

    # Append any remaining table in progress to core after the last page
    if not table_in_progress.empty:
        print(f"Appending final table in progress to core")
        core = pd.concat([core, table_in_progress], ignore_index=True)

# Escape special characters in the headers to avoid regex issues
escaped_headers = [re.escape(header) for header in headers]    

'''

def clean_string_cell(value):
    if isinstance(value, str):
        return value.replace('\n', ' ').strip()  # Remove newlines and extra spaces
    return value  # Return the original value if it's not a string

core = core.map(clean_string_cell)

# Step 2: Drop rows that contain specific phrases (e.g., "Type of Upgrade")
core = core[~core.apply(lambda row: row.astype(str).str.contains('Type of Upgrade', case=False, na=False).any(), axis=1)]

'''

def contains_phrase(row, phrase):
    # Build the regex pattern dynamically, allowing for any whitespace (\s*) between words
    regex_pattern = re.sub(r"\s+", r"\\s*", phrase)
    pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
    return row.astype(str).apply(lambda cell: bool(pattern.search(cell))).any()

# Apply the regex-based row filter
core = core[~core.apply(lambda row: contains_phrase(row, "Type of Upgrade"), axis=1)]



# Ensure the final DataFrame has all the columns detected (dynamically add new columns)
#core = core.reindex(columns=pd.Index(sorted(core.columns.unique())))

# Print the final combined DataFrame
 
print(tabulate(core, headers='keys', tablefmt='fancy_grid', showindex=False))

# If you want to export this to a CSV file, uncomment the following line:
# core.to_csv('final_table_data.csv', index=False)


New table detected on page 10, table 1 with 5 columns
Detected continuation on page 10, table 2
Detected continuation on page 11, table 1
Appending previous table to core from page 11
New table detected on page 11, table 2 with 7 columns
Appending final table in progress to core
╒═══════════════════╤═══════════════════════════════════════════════╤══════════════════════════╤═══════════════════════════════════╤═══════════════════════════════╤═════════════════════════════════════════╤═════════════════════════╤════════════════╕
│ type of upgrade   │ upgrade                                       │ cost allocation factor   │ estimated cost x 1000 escalated   │ estimated time to construct   │ total estimated cost x 1000 escalated   │ adnu cost rate x 1000   │   project size │
╞═══════════════════╪═══════════════════════════════════════════════╪══════════════════════════╪═══════════════════════════════════╪═══════════════════════════════╪═════════════════════════════════════════╪══════════════

# This is the penultimate one

In [46]:
import os
import pdfplumber
import pandas as pd
import re
from tabulate import tabulate

#new_directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/matched_pdfs_phase_i/"
#os.chdir(new_directory)

# List of PDFs to process
#pdfs= ["Q1030/C7PhI - Appendix A - Q1030   South Lake Solar.pdf",
 #       "Q1040/QC7PhI_Q1040_Arlington Valley Solar Energy 2_Appendix A_12-17-2014.pdf"]

pdfs = [
    "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1030/02_phase_1_study/C7PhI - Appendix A - Q1030   South Lake Solar.pdf",
   "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1158/02_phase_1_study/Q1158 Slate_Appendix A Report_C8PhI.pdf",
]

# Initialize an empty DataFrame to store all tables
core = pd.DataFrame()

# Function to clean up and standardize column headers, including removing brackets and their contents
def clean_column_headers(headers):
    cleaned_headers = []
    for header in headers:
        if header is None:  # Handle None type values
            header = ""
        elif isinstance(header, str):  # Only process if it's a string
            header = header.lower()  # Convert to lowercase
            header = re.sub(r'\s+', ' ', header)  # Replace multiple spaces with a single space
            header = re.sub(r'\(.*?\)', '', header)  # Remove anything in parentheses, including brackets
            header = re.sub(r'[^a-zA-Z0-9\s]', '', header)  # Remove special characters like $ and commas
            header = header.strip()  # Remove leading/trailing spaces
        cleaned_headers.append(header)
    return cleaned_headers

# Function to clean string cells in the table
def clean_string_cell(value):
    if isinstance(value, str):
        return value.replace('\n', ' ').strip()  # Remove newlines and extra spaces
    return value  # Return the original value if it's not a string

# Function to check if a row contains a specific phrase
def contains_phrase(row, phrase):
    # Build the regex pattern dynamically, allowing for any whitespace (\s*) between words
    regex_pattern = re.sub(r"\s+", r"\\s*", phrase)
    pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
    return row.astype(str).apply(lambda cell: bool(pattern.search(cell))).any()


# Function to find the table title based on the identifier (e.g., "Table 7")
def find_table_title(page_text, table_identifier):
    table_title_match = re.search(rf"({table_identifier}[^\n]+)", page_text)
    if table_title_match:
        return table_title_match.group(1)
    return None

# Function to search for specific patterns in the table title (e.g., "PTO", "Reliability Network Upgrade")
def search_in_table_title(table_title, patterns):
    for pattern in patterns:
        if re.search(pattern, table_title, re.IGNORECASE):
            return pattern
    return ""


# Process each PDF in the list
for pdf_path in pdfs:
    print(f"Processing {pdf_path}...")

    with pdfplumber.open(pdf_path) as pdf:
        table_in_progress = pd.DataFrame()
        headers = None  # To store headers of the table that starts on the first page
        expected_columns = None  # To track the number of columns expected for a continuation
        title_pattern = None 
        
        for page_number, page in enumerate(pdf.pages):
            text = page.extract_text()

            # Only process pages with "Table 7" in the text
            if re.search(r"Table 7", text):
                # Extract all tables from the page
                tables = page.find_tables(table_settings={
                    "horizontal_strategy": "lines",
                    "vertical_strategy": "lines",
                })
                # Find the table title on the page

                

                if not title_pattern:
                    table_title = find_table_title(text, "Table 7")
                    # Search for a specific pattern in the table title
                    title_pattern = search_in_table_title(table_title, ["PTO", "Reliability Network Upgrade"])
                    print(f"New title pattern detected: {title_pattern}")

                # If no tables found on the page, continue to the next page
                if not tables:
                    print(f"No tables found on page {page_number + 1}")
                    continue

                # Iterate over each table on the page
                for table_index, table in enumerate(tables):
                    tab = table.extract()

                    # If the table is empty, skip it
                    if len(tab) == 0:
                        continue

                    # Extract headers and rows
                    current_headers = tab[0]  # First row as headers
                    cleaned_headers = clean_column_headers(current_headers)  # Clean headers
                    data_rows = tab[1:]  # Data rows

                    # Check if the "Type of Upgrade" column (index 0) has missing values
                    # and replace only the first occurrence with the title pattern, until the "total" row is encountered
                    # Logic to handle missing entries in "Type of Upgrade"
                    replaced = False  # Only replace the first instance of None or empty
                    for row in data_rows:
                        # Check if any cell in the row contains "Total" (handling different cases and trimming whitespace)
                        if any(cell is not None and isinstance(cell, str) and "Total" in cell.strip() for cell in row):
                            print(f"'Total' found in row: {row}")  # Debugging print statement
                            break  # Stop replacing when "Total" row is encountered

                        # Replace the first None or empty value only in the first column
                        if not replaced and (row[0] is None or row[0].strip() == ""):
                            print(f"Replacing None/empty in row: {row}")  # Debugging print statement
                            row[0] = title_pattern  # Replace with title pattern
                            replaced = True  # Ensure only first occurrence is replaced



                    # If the table has the same number of columns, consider it a continuation
                    if headers and len(current_headers) == expected_columns:
                        print(f"Detected continuation on page {page_number + 1}, table {table_index + 1}")
                        continuation_data = pd.DataFrame(tab, columns=headers)
                        table_in_progress = pd.concat([table_in_progress, continuation_data], ignore_index=True)
                    else:
                        # If a new table starts, append the old table in progress
                        if not table_in_progress.empty:
                            print(f"Appending previous table to core from page {page_number + 1}")
                            core = pd.concat([core, table_in_progress], ignore_index=True)
                            table_in_progress = pd.DataFrame()  # Reset the in-progress table

                        # Start a new table
                        headers = cleaned_headers
                        expected_columns = len(headers)
                        print(f"New table detected on page {page_number + 1}, table {table_index + 1} with {expected_columns} columns")
                        table_in_progress = pd.DataFrame(data_rows, columns=headers)

                        title_pattern = None

        # Append any remaining table in progress to core after the last page
        if not table_in_progress.empty:
            print(f"Appending final table in progress to core for {pdf_path}")
            core = pd.concat([core, table_in_progress], ignore_index=True)

# Clean up the entire DataFrame by cleaning string cells
core = core.map(clean_string_cell)

# Drop rows that contain specific phrases (e.g., "Type of Upgrade")
core = core[~core.apply(lambda row: contains_phrase(row, "Type of Upgrade"), axis=1)]

# Ensure the final DataFrame has all the columns detected (dynamically add new columns)
#core = core.reindex(columns=pd.Index(sorted(core.columns.unique())))




## Merge Columns:
# Merge columns that have similar names or convey the same information

# Example 1: Merge "cost x 1000", "cost x1,000", "cost x$1000"
#columns_to_merge = ["escalated costs x 1000", "estimated cost x 1000 escalated", ]
#core["estimated cost x 1000"] = core[columns_to_merge].bfill(axis=1).iloc[:, 0]
#core = core.drop(columns=columns_to_merge[1:])

# Example 2: Merge "expected costs" and "expected costs (Note 1)"
#columns_to_merge = ["estimated time  to construct", "estimated time to construct"]
#core["estimated time to construct"] = core[columns_to_merge].bfill(axis=1).iloc[:, 0]
#core = core.drop(columns=columns_to_merge[1:])

# Print the final combined DataFrame
print(tabulate(core, headers='keys', tablefmt='fancy_grid', showindex=False))

# If you want to export this to a CSV file, uncomment the following line:
core.to_csv('traildata.csv', index=False)


Processing /Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/1030/02_phase_1_study/C7PhI - Appendix A - Q1030   South Lake Solar.pdf...
New title pattern detected: 
Replacing None/empty in row: [None, '70kV Switching Station', '• Gen – Tie Line Relays, Dead-\nend, PTs, and Disconnect\nSwitch\n• Install Fiber Terminations', '100.00%', '$720', '$789', '24']
'Total' found in row: ['', '', '', 'Total', '$1,589', '$1,741', '']
New table detected on page 13, table 1 with 7 columns
New title pattern detected: Reliability Network Upgrade
Replacing None/empty in row: [None, 'Schindler Substation', '• New DTT Transmitter, Channel\nBank, & HVP Installation (by\nQ532)', '100.00%', 'N/A', 'N/A', '24']
Detected continuation on page 14, table 1
Replacing None/empty in row: [None, '70kV Switching Station', '• One (1) 70kV CB, Dead-end,\nSwitches, and Breaker\nProtection Package.', '100.00%', '$900', '$986', '24']
'Total' found in row: ['', '', '', 'Total', '$1,006', '$1,103', '']
Detect

# This works and fixes issues with the skipping pages in clusters 6,7,8

In [66]:
import os
import pdfplumber
import pandas as pd
import re
import PyPDF2
from tabulate import tabulate

pdfs = [
    "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1030/02_phase_1_study/C7PhI - Appendix A - Q1030   South Lake Solar.pdf",
    "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1035/02_phase_1_study/C7PhI Appendix A - Q1035 Magellan Solar Final.pdf",
    "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1130/02_phase_1_study/Q1130 Moon Prism 2_Appendix A Report_C8PhI.pdf",
    "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1158/02_phase_1_study/Q1158 Slate_Appendix A Report_C8PhI.pdf",
    "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1170/02_phase_1_study/QC8PhI_Q1170_Gateway Energy Storage_Appendix A_1-15-2016.pdf"
]

# Initialize an empty DataFrame to store all tables
core = pd.DataFrame()

def clean_column_headers(headers):
    cleaned_headers = []
    for header in headers:
        if header is None:
            header = ""
        elif isinstance(header, str):
            header = header.lower()
            header = re.sub(r'\s+', ' ', header)
            header = re.sub(r'\(.*?\)', '', header)
            header = re.sub(r'[^a-zA-Z0-9\s]', '', header)
            header = header.strip()
        cleaned_headers.append(header)
    return cleaned_headers

def clean_string_cell(value):
    if isinstance(value, str):
        return value.replace('\n', ' ').strip()
    return value

def contains_phrase(row, phrase):
    regex_pattern = re.sub(r"\s+", r"\\s*", phrase)
    pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
    return row.astype(str).apply(lambda cell: bool(pattern.search(cell))).any()

def find_table_title(page_text, table_identifier):
    table_title_match = re.search(rf"({table_identifier}[^\n]+)", page_text)
    if table_title_match:
        return table_title_match.group(1)
    return None

def search_in_table_title(table_title, patterns):
    for pattern in patterns:
        if re.search(pattern, table_title, re.IGNORECASE):
            return pattern
    return table_title  # Return the entire title if no pattern matches

def extract_point_of_interconnection(pdf_path):
    """Extract the 'Point of Interconnection' from 'Table 1-', handling multi-page tables."""
    print("Extracting Point of Interconnection...")
    with pdfplumber.open(pdf_path) as pdf:
        table_in_progress = pd.DataFrame()
        headers = None
        expected_columns = None
        in_table1 = False

        for page_number in range(len(pdf.pages)):
            page = pdf.pages[page_number]
            text = page.extract_text()

            # Check if 'Table 1-' is in the text
            if re.search(r"Table\s*1", text, re.IGNORECASE) and not in_table1:
                print(f"Found 'Table 1' on page {page_number + 1}")
                in_table1 = True
                replaced = False  # Reset 'replaced' when starting a new table

            if in_table1:
                print(f"Processing page {page_number + 1} for 'Table 1'...")
                tables = page.extract_tables(table_settings={
                    "vertical_strategy": "lines",
                    "horizontal_strategy": "lines",
                })

                if not tables:
                    print(f"No tables found on page {page_number + 1}")
                    in_table1 = False
                    continue

                for table_index, table in enumerate(tables):
                    tab = table

                    if len(tab) == 0:
                        continue

                    # Determine if this is a continuation table
                    if headers and len(tab[0]) == expected_columns:
                        # Continuation table
                        print(f"Detected continuation on page {page_number + 1}, table {table_index + 1}")
                        # Check if the first row matches the headers, skip if it does
                        first_row = tab[0]
                        cleaned_first_row = clean_column_headers(first_row)
                        if cleaned_first_row == headers:
                            data_rows = tab[1:]  # Skip the first row
                        else:
                            data_rows = tab  # All rows are data
                    else:
                        # New table
                        current_headers = tab[0]
                        cleaned_headers = clean_column_headers(current_headers)
                        headers = cleaned_headers
                        expected_columns = len(headers)
                        data_rows = tab[1:]  # Skip the header row
                        print(f"New table headers: {headers}")

                    # Create DataFrame for current data_rows
                    if headers and data_rows:
                        df_data = pd.DataFrame(data_rows, columns=headers)
                        table_in_progress = pd.concat([table_in_progress, df_data], ignore_index=True)
                    else:
                        print(f"Data row length mismatch on page {page_number + 1}, table {table_index + 1}")

                # Check if the next page likely continues Table 1
                if page_number + 1 < len(pdf.pages):
                    next_page = pdf.pages[page_number + 1]
                    next_text = next_page.extract_text()
                    if re.search(r"Table\s*\d", next_text, re.IGNORECASE) and not re.search(r"Table\s*1", next_text, re.IGNORECASE):
                        in_table1 = False
                        print(f"Found new table on page {page_number + 2}, ending Table 1 extraction.")
                else:
                    in_table1 = False
            else:
                continue

        if table_in_progress.empty:
            print("No data extracted from Table 1.")
            return None

        # Clean the DataFrame
        table_in_progress = table_in_progress.applymap(clean_string_cell)

        # Search for 'Point of Interconnection' in the DataFrame
        for idx, row in table_in_progress.iterrows():
            if row.astype(str).str.contains("point of interconnection", case=False, regex=False).any():
                print(f"Found 'Point of Interconnection' in row {idx + 1}")
                poi_index = row.index[row.astype(str).str.contains("point of interconnection", case=False, regex=False)][0]
                poi_index_loc = row.index.get_loc(poi_index)
                if poi_index_loc + 1 < len(row):
                    poi_value = row.iloc[poi_index_loc + 1]
                    print(f"Extracted Point of Interconnection: {poi_value}")
                    return poi_value.strip()
                else:
                    print("Adjacent cell for 'Point of Interconnection' not found.")
                    return None

        print("Point of Interconnection not found in Table 1.")
        return None






def search_gps_coordinates(text):
    """Search for GPS coordinates using multiple patterns."""
    print("Searching for GPS coordinates...")
    gps_coords = re.search(r"gps coordinates:\s*([\d\.\-]+),\s*([\d\.\-]+)", text, re.IGNORECASE)
    if gps_coords:
        print(f"Found GPS coordinates: {gps_coords.groups()}")
        return gps_coords.groups()

    project_coords = re.search(r"latitude[:\s]*([\d\.\-]+)[^\d]+longitude[:\s]*([\d\.\-]+)", text, re.IGNORECASE)
    if project_coords:
        print(f"Found project coordinates: {project_coords.groups()}")
        return project_coords.groups()

    gps_coords_directional = re.search(
        r"gps coordinates:\s*([\d\.\-]+)\s*[nNsS],\s*([\d\.\-]+)\s*[eEwW]", text, re.IGNORECASE)
    if gps_coords_directional:
        lat, lon = gps_coords_directional.groups()
        latitude = lat if "N" in text.upper() else f"-{lat}"  # Adjust latitude sign
        longitude = lon if "E" in text.upper() else f"-{lon}"  # Adjust longitude sign
        print(f"Found directional GPS coordinates: {(latitude, longitude)}")
        return (latitude, longitude)

    print("GPS coordinates not found.")
    return (None, None)

def extract_base_data(pdf_path, project_id):
    """Extract base data from the PDF and return as a DataFrame."""
    print("Extracting base data from PDF...")
    with open(pdf_path, 'rb') as pdf:
        reader = PyPDF2.PdfReader(pdf)
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text

    text = clean_string_cell(text)

    queue_id = re.search(r"q[\s_-]*(\d+)", text, re.IGNORECASE)
    queue_id = queue_id.group(1) if queue_id else str(project_id)  # Use project_id if queue_id is not found
    print(f"Extracted Queue ID: {queue_id}")

    cluster_number = re.search(r"queue[\s_-]*cluster[\s_-]*(\d+)", text, re.IGNORECASE)
    cluster_number = cluster_number.group(1) if cluster_number else None
    print(f"Extracted Cluster Number: {cluster_number}")

    deliverability_status = re.search(r"(\w+)\s*capacity deliverability status", text, re.IGNORECASE)
    deliverability_status = deliverability_status.group(1) if deliverability_status else None
    print(f"Extracted Deliverability Status: {deliverability_status}")

    # Search for GPS coordinates
    latitude, longitude = search_gps_coordinates(text)

    capacity = re.search(r"total rated output of (\d+)\s*mw", text, re.IGNORECASE)
    if capacity:
        capacity = int(capacity.group(1))
    else:
        capacity2 = re.search(r"(\d+)\s*mw", text)
        capacity = int(capacity2.group(1)) if capacity2 else None
    print(f"Extracted Capacity: {capacity}")

    # Extract Point of Interconnection
    point_of_interconnection = extract_point_of_interconnection(pdf_path)

    # Initialize base data dictionary
    base_data = {
        "q_id": [queue_id],
        "cluster": [cluster_number],
        "req_deliverability": [deliverability_status],
        "latitude": [latitude],
        "longitude": [longitude],
        "capacity": [capacity],
        "point_of_interconnection": [point_of_interconnection]
    }

    print("Base data extracted:", base_data)
    return pd.DataFrame(base_data)

# Process each PDF in the list
for pdf_path in pdfs:
    print(f"\nProcessing {pdf_path}...")

    # Extract project_id from pdf_path if needed
    project_id = os.path.basename(pdf_path).split('.')[0]  # Modify as appropriate

    # Extract base data
    base_data = extract_base_data(pdf_path, project_id)

    # Initialize an empty DataFrame to store tables from this PDF
    pdf_core = pd.DataFrame()

    with pdfplumber.open(pdf_path) as pdf:
        table_in_progress = pd.DataFrame()
        headers = None
        expected_columns = None
        title_pattern = None
        in_table7 = False
        replaced = False  # Initialize 'replaced' here
        previous_title_pattern = None  # Keep track of previous table title

        for page_number, page in enumerate(pdf.pages):
            text = page.extract_text()

            # Check if "Table 7" is in the text
            if re.search(r"Table 7", text):
                in_table7 = True
                table_title = find_table_title(text, "Table 7")
                title_pattern = search_in_table_title(table_title, ["PTO", "Reliability Network Upgrade", "Area Delivery Network Upgrade", "Local Delivery Network"])
                print(f"New title pattern detected: {title_pattern}")
                previous_title_pattern = title_pattern  # Update previous title pattern
                replaced = False  # Reset 'replaced' when starting a new table

            if in_table7:
                # Extract all tables from the page
                tables = page.find_tables(table_settings={
                    "horizontal_strategy": "lines",
                    "vertical_strategy": "lines",
                })

                if not tables:
                    print(f"No tables found on page {page_number + 1}")
                    in_table7 = False  # Assume table has ended
                    continue

                for table_index, table in enumerate(tables):
                    tab = table.extract()

                    if len(tab) == 0:
                        continue

                    # Determine if this is a continuation table
                    if headers and len(tab[0]) == expected_columns:
                        # Continuation table
                        print(f"Detected continuation on page {page_number + 1}, table {table_index + 1}")
                        # Check if the first row matches the headers, skip if it does
                        first_row = tab[0]
                        cleaned_first_row = clean_column_headers(first_row)
                        if cleaned_first_row == headers:
                            data_rows = tab[1:]  # Skip the first row

                            if data_rows and not replaced:
                                for idx, row in enumerate(data_rows):
                                    if row[0] is None or row[0].strip() == "":
                                        print(f"Setting 'Type of Upgrade' in data row: {row}")
                                        row[0] = previous_title_pattern  # Use previous title pattern
                                        replaced = True
                                        break  # Only replace once per table
                        else:
                            data_rows = tab  # All rows are data
                    else:
                        # New table
                        if not table_in_progress.empty:
                            print(f"Appending previous table to pdf_core from page {page_number + 1}")
                            pdf_core = pd.concat([pdf_core, table_in_progress], ignore_index=True)
                            table_in_progress = pd.DataFrame()
                        current_headers = tab[0]
                        cleaned_headers = clean_column_headers(current_headers)
                        data_rows = tab[1:]
                        headers = cleaned_headers
                        expected_columns = len(headers)
                        print(f"New table detected on page {page_number + 1}, table {table_index + 1} with {expected_columns} columns")
                        replaced = False  # Reset 'replaced' when starting a new table

                    # Create DataFrame for current data_rows
                    if headers and data_rows and len(data_rows[0]) == expected_columns:
                        df_data = pd.DataFrame(data_rows, columns=headers)
                        table_in_progress = pd.concat([table_in_progress, df_data], ignore_index=True)
                    else:
                        print(f"Data row length mismatch on page {page_number + 1}, table {table_index + 1}")
            else:
                continue

        # Append any remaining table in progress to pdf_core after the last page
        if not table_in_progress.empty:
            print(f"Appending final table in progress to pdf_core for {pdf_path}")
            pdf_core = pd.concat([pdf_core, table_in_progress], ignore_index=True)

    # Clean up the entire DataFrame
    pdf_core = pdf_core.applymap(clean_string_cell)

    # Drop rows that contain specific phrases
    pdf_core = pdf_core[~pdf_core.apply(lambda row: contains_phrase(row, "Type of Upgrade"), axis=1)]

    # Merge base data with pdf_core
    if not pdf_core.empty:
        overlapping_columns = base_data.columns.intersection(pdf_core.columns)
        pdf_core = pdf_core.drop(columns=overlapping_columns)
        base_data_repeated = pd.concat([base_data] * len(pdf_core), ignore_index=True)
        merged_df = pd.concat([base_data_repeated, pdf_core], axis=1)
        print(f"Merged base data with table data for {pdf_path}")
    else:
        # If no table data extracted, use base data only
        merged_df = base_data
        print(f"No Table 7 data found in {pdf_path}. Using base data only.")

    # Append merged_df to core
    core = pd.concat([core, merged_df], ignore_index=True)

# Display the final combined DataFrame
print("\nFinal combined DataFrame:")
print(tabulate(core, headers='keys', tablefmt='fancy_grid', showindex=False))

# Save to CSV if needed
core.to_csv('final_data.csv', index=False)
 



Processing /Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/1030/02_phase_1_study/C7PhI - Appendix A - Q1030   South Lake Solar.pdf...
Extracting base data from PDF...
Extracted Queue ID: 1030
Extracted Cluster Number: 7
Extracted Deliverability Status: Full
Searching for GPS coordinates...
Found directional GPS coordinates: ('36.379', '120.138')
Extracted Capacity: 20
Extracting Point of Interconnection...
Found 'Table 1' on page 4
Processing page 4 for 'Table 1'...
No tables found on page 4
Found 'Table 1' on page 5
Processing page 5 for 'Table 1'...
New table headers: ['project parameters', 'project specific data']
Processing page 6 for 'Table 1'...
No tables found on page 6


/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_4857/1141684912.py:136: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  table_in_progress = table_in_progress.applymap(clean_string_cell)


Found 'Point of Interconnection' in row 10
Extracted Point of Interconnection: Schindler-Huron-Gates tap 70kV
Base data extracted: {'q_id': ['1030'], 'cluster': ['7'], 'req_deliverability': ['Full'], 'latitude': ['36.379'], 'longitude': ['120.138'], 'capacity': [20], 'point_of_interconnection': ['Schindler-Huron-Gates tap 70kV']}
New title pattern detected: Table 7-1: Escalated Cost and Time to Construct for Interconnection Facilities - IF
New table detected on page 13, table 1 with 7 columns
New title pattern detected: Reliability Network Upgrade
Detected continuation on page 14, table 1
Setting 'Type of Upgrade' in data row: [None, 'Schindler Substation', '• New DTT Transmitter, Channel\nBank, & HVP Installation (by\nQ532)', '100.00%', 'N/A', 'N/A', '24']
New title pattern detected: Local Delivery Network
Detected continuation on page 15, table 1
Detected continuation on page 15, table 2
New title pattern detected: Area Delivery Network Upgrade
Detected continuation on page 16, table

/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_4857/1141684912.py:338: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pdf_core = pdf_core.applymap(clean_string_cell)


Extracted Queue ID: 1035
Extracted Cluster Number: 7
Extracted Deliverability Status: Full
Searching for GPS coordinates...
GPS coordinates not found.
Extracted Capacity: 20
Extracting Point of Interconnection...
Found 'Table 1' on page 4
Processing page 4 for 'Table 1'...
New table headers: ['project parameters', 'project specific data']
Processing page 5 for 'Table 1'...
Detected continuation on page 5, table 1
Processing page 6 for 'Table 1'...
No tables found on page 6


...

/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_4857/1141684912.py:338: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pdf_core = pdf_core.applymap(clean_string_cell)


Extracted Queue ID: 1170
Extracted Cluster Number: 8
Extracted Deliverability Status: None
Searching for GPS coordinates...
GPS coordinates not found.
Extracted Capacity: None
Extracting Point of Interconnection...
Found 'Table 1' on page 4
Processing page 4 for 'Table 1'...
New table headers: ['project location', 'tbd near otay mesa 230 kv switchyard']
Processing page 5 for 'Table 1'...
No tables found on page 5


/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_4857/1141684912.py:136: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  table_in_progress = table_in_progress.applymap(clean_string_cell)


Found 'Point of Interconnection' in row 9
Extracted Point of Interconnection: Otay Mesa switchyard 230 kV bus
Base data extracted: {'q_id': ['1170'], 'cluster': ['8'], 'req_deliverability': [None], 'latitude': [None], 'longitude': [None], 'capacity': [None], 'point_of_interconnection': ['Otay Mesa switchyard 230 kV bus']}
New title pattern detected: PTO
New table detected on page 11, table 1 with 7 columns
New title pattern detected: Table 7.2: RNUs, Estimated Costs, and Estimated Time to Construct Summary
Appending previous table to pdf_core from page 12
New table detected on page 12, table 1 with 6 columns
New title pattern detected: Table 7.3: LDNUs Estimated Costs, Cost Rate, Estimated Time to Construct
Appending previous table to pdf_core from page 13
New table detected on page 13, table 1 with 5 columns
Appending previous table to pdf_core from page 13
New table detected on page 13, table 2 with 7 columns
No tables found on page 14
Appending final table in progress to pdf_core fo

/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_4857/1141684912.py:338: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pdf_core = pdf_core.applymap(clean_string_cell)


# trying out a problem in clusrter 8

# Lets try for all cluster 7

In [303]:
import os
import pdfplumber
import pandas as pd
import re
import PyPDF2
from tabulate import tabulate
import csv
import unicodedata

class Cluster7:
    def __init__(self, directory, output_csv_path):
        self.directory = directory
        self.output_csv_path = output_csv_path
        self.core = pd.DataFrame()  # Initialize an empty DataFrame to store all tables
        self.cluster_7_patterns = [
            r"C[\s_-]*7", r"c[\s_-]*7", r"Cluster[\s_-]*7", r"cluster[\s_-]*7", 
            r"C[\s_-]*VII", r"c[\s_-]*VII", r"Cluster[\s_-]*VII", r"cluster[\s_-]*VII"
        ]
        self.pdf_count = 0
        self.scraped_count = 0

    def clean_column_headers(self, headers):
        cleaned_headers = []
        for header in headers:
            if header is None:
                header = ""
            elif isinstance(header, str):
                header = header.lower()
                header = re.sub(r'\s+', ' ', header)
                header = re.sub(r'\(.*?\)', '', header)  # Remove anything in parentheses
                header = re.sub(r'[^a-zA-Z0-9\s]', '', header)
                header = header.strip()
            cleaned_headers.append(header)
        return cleaned_headers

    def clean_string_cell(self, value):
        if isinstance(value, str):
            # Normalize Unicode to NFKD and remove non-ASCII characters
            value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
            # Remove newlines and extra spaces
            value = value.replace('\n', ' ').strip()
        return value
        

    def contains_phrase(self, row, phrase):
        regex_pattern = re.sub(r"\s+", r"\\s*", phrase)
        pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
        return row.astype(str).apply(lambda cell: bool(pattern.search(cell))).any()
    
    
    def merge_columns(self, df):
        """Merge columns that have similar names or convey the same information."""
        merge_columns = {
            "escalated cost x 1000": ["escalated costs x 1000", "estimated cost x 1000 escalated"],
            "estimated time to construct": ["estimated time to construct", "estimated time  to construct"],
            "total estimated cost x 1000 escalalted": ["total estimated cost x 1000 escalalted", "total estimated cost x 1000 escalated"],
            "adnu cost rate x 1000": ["adnu cost rate x 1000", "cost rate x 1000"],
            "description": ["description"],
        }

        # Identify unnamed columns (empty or NaN header)
        unnamed_columns = [col for col in df.columns if pd.isna(col) or col.strip() == ""]
        
        if unnamed_columns:
            # Merge unnamed columns into 'description' if they exist
            merge_columns["description"].extend(unnamed_columns)
        
        for new_col, old_cols in merge_columns.items():
            if any(col in df.columns for col in old_cols):
                df[new_col] = df[old_cols].bfill(axis=1).iloc[:, 0]
                df.drop(columns=[col for col in old_cols if col in df.columns and col != new_col], inplace=True)
        
        return df
      

    def extract_point_of_interconnection(self, pdf_path):
        """Extract the 'Point of Interconnection' from either Table 1 or using regex."""
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if "Table 1-" in page_text:
                    tables = page.extract_tables(table_settings={
                        "horizontal_strategy": "lines",
                        "vertical_strategy": "lines"
                    })
                    for table in tables:
                        for row in table:
                            if row and "Point of Interconnection" in row[0]:
                                return row[1]

        # Fallback to regex search in the text
        with open(pdf_path, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in reader.pages:
                text += page.extract_text()

        point_of_interconnection_match = re.search(r"point of interconnection[:\s]*(.+)", text, re.IGNORECASE)
        if point_of_interconnection_match:
            return point_of_interconnection_match.group(1).strip()

        return None

    def search_gps_coordinates(self, text):
        """Search for GPS coordinates using multiple patterns."""
        gps_coords = re.search(r"gps coordinates:\s*([\d\.\-]+),\s*([\d\.\-]+)", text, re.IGNORECASE)
        if gps_coords:
            return gps_coords.groups()

        project_coords = re.search(r"latitude[:\s]*([\d\.\-]+)[^\d]+longitude[:\s]*([\d\.\-]+)", text, re.IGNORECASE)
        if project_coords:
            return project_coords.groups()

        gps_coords_directional = re.search(
            r"gps coordinates:\s*([\d\.\-]+)\s*[nNsS],\s*([\d\.\-]+)\s*[eEwW]", text, re.IGNORECASE)
        if gps_coords_directional:
            lat, lon = gps_coords_directional.groups()
            latitude = lat if "N" in text.upper() else f"-{lat}"  # Adjust latitude sign
            longitude = lon if "E" in text.upper() else f"-{lon}"  # Adjust longitude sign
            return (latitude, longitude)

        return (None, None)

    def extract_pdf_data(self, pdf_path):
        """Extract base data from the PDF and return as a DataFrame."""
        with open(pdf_path, 'rb') as pdf:
            reader = PyPDF2.PdfReader(pdf)
            text = ""
            for page in reader.pages:
                text += page.extract_text()

        text = self.clean_string_cell(text)

        queue_id = re.search(r"q[\s_-]*(\d+)", text, re.IGNORECASE)
        queue_id = queue_id.group(1) if queue_id else None

        cluster_number = re.search(r"queue[\s_-]*cluster[\s_-]*(\d+)", text)
        cluster_number = cluster_number.group(1) if cluster_number else "7"

        deliverability_status = re.search(r"(\w+)\s*capacity deliverability status", text, re.IGNORECASE)
        deliverability_status = deliverability_status.group(1) if deliverability_status else None

        # Search for GPS coordinates
        latitude, longitude = self.search_gps_coordinates(text)

        capacity = re.search(r"total rated output of (\d+)\s*mw", text, re.IGNORECASE)
        if capacity:
            capacity = int(capacity.group(1))
        else:
            capacity2 = re.search(r"(\d+)\s*mw", text)
            capacity = int(capacity2.group(1)) if capacity2 else None

        # Extract Point of Interconnection
        point_of_interconnection = self.extract_point_of_interconnection(pdf_path)

        # Initialize base data dictionary
        base_data = {
            "q_id": [queue_id],
            "cluster": [cluster_number],
            "req_deliverability": [deliverability_status],
            "latitude": [latitude],
            "longitude": [longitude],
            "capacity": [capacity],
            "point_of_interconnection": [point_of_interconnection]
        }

        return pd.DataFrame(base_data)

    def extract_table7_and_replace_none(self, pdf_path):
        base_data = self.extract_pdf_data(pdf_path)
        pdf_core = pd.DataFrame()

        with pdfplumber.open(pdf_path) as pdf:
            table_in_progress = pd.DataFrame()
            headers = None
            expected_columns = None
            title_pattern = None
            last_known_upgrade = None 

            for page_number, page in enumerate(pdf.pages):
                text = page.extract_text()

                if re.search(r"Table 7", text):
                    tables = page.find_tables(table_settings={
                        "horizontal_strategy": "lines",
                        "vertical_strategy": "lines",
                    })

                    if not title_pattern:
                        table_title = self.find_table_title(text, "Table 7")
                        title_pattern = self.search_in_table_title(table_title, ["PTO", "Reliability Network Upgrade"])

                    if not tables:
                        continue

                    for table_index, table in enumerate(tables):
                        tab = table.extract()
                        if len(tab) == 0:
                            continue

                        current_headers = tab[0]
                        cleaned_headers = self.clean_column_headers(current_headers)
                        data_rows = tab[1:]

                        replaced = False
                        for row in data_rows:
                            if any(cell and "Total" in cell for cell in row):
                               break
                            '''
                            if not replaced and (row[0] is None or row[0].strip() == ""):
                                row[0] = title_pattern
                                replaced = True
                            '''
                            if row[0] is None or row[0].strip() == "":
                                # If no previous value, use the title_pattern as default
                                if last_known_upgrade is None:
                                    row[0] = title_pattern
                                else:
                                    row[0] = last_known_upgrade  # Use the last known non-None value
                            else:
                                # Update last_known_upgrade with the current non-None value
                                last_known_upgrade = row[0]
                                
                        if headers and len(current_headers) == expected_columns:
                            continuation_data = pd.DataFrame(tab, columns=headers)
                            table_in_progress = pd.concat([table_in_progress, continuation_data], ignore_index=True)
                        else:
                            if not table_in_progress.empty:
                                pdf_core = pd.concat([pdf_core, table_in_progress], ignore_index=True)
                                table_in_progress = pd.DataFrame()

                            headers = cleaned_headers
                            expected_columns = len(headers)
                            table_in_progress = pd.DataFrame(data_rows, columns=headers)

            if not table_in_progress.empty:
                pdf_core = pd.concat([pdf_core, table_in_progress], ignore_index=True)

        overlapping_columns = base_data.columns.intersection(pdf_core.columns)
        pdf_core = pdf_core.drop(columns=overlapping_columns)
        merged_df = pd.concat([base_data] * len(pdf_core), ignore_index=True).join(pdf_core)

        return merged_df

    def process_pdfs_in_folder(self):
        for root, dirs, files in os.walk(self.directory):
            for pdf_name in files:
                if pdf_name.endswith(".pdf") and any(re.search(pattern, pdf_name, re.IGNORECASE) for pattern in self.cluster_7_patterns):
                    pdf_path = os.path.join(root, pdf_name)
                    self.pdf_count += 1
                    print(f"Processing {pdf_name}")

                    try:
                        df = self.extract_table7_and_replace_none(pdf_path)
                        if not df.empty:
                            self.core = pd.concat([self.core, df], ignore_index=True)
                            self.scraped_count += 1
                            print(f"Scraped data from {pdf_name}")
                    except Exception as e:
                        print(f"Skipping {pdf_name} due to an error: {e}")


    def save_to_csv(self):
        # Clean up the entire DataFrame by cleaning string cells
        self.core = self.core.map(self.clean_string_cell)

        # Drop rows that contain specific phrases (e.g., "Type of Upgrade")
        self.core = self.core[~self.core.apply(lambda row: self.contains_phrase(row, "Type of Upgrade"), axis=1)]

        # Merge similar or duplicate columns
        self.core = self.merge_columns(self.core)

        # Ensure q_id is numeric for sorting, replace missing values with None
        self.core['q_id'] = pd.to_numeric(self.core['q_id'], errors='coerce')

        # Save the unsorted DataFrame to CSV first
        self.core.to_csv(self.output_csv_path, index=False, encoding='utf-8', sep=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

        # Sort by 'q_id' after saving and reload the file to avoid issues with sorting
        #sorted_data = self.core.sort_values(by="q_id", ascending=True, na_position='last')
        #sorted_data.to_csv(self.output_csv_path, index=False)

        print(f"Data extracted and saved to {self.output_csv_path}")
        print(f"Total PDFs accessed: {self.pdf_count}")
        print(f"Total PDFs successfully scraped: {self.scraped_count}")
                    
    '''  
    def save_to_csv(self):
        self.core = self.core.applymap(self.clean_string_cell)
        self.core = self.core[~self.core.apply(lambda row: self.contains_phrase(row, "Type of Upgrade"), axis=1)]
        # Sort by q_id in ascending order
        #self.core = self.core.sort_values(by="q_id", ascending=True)

        # Merge similar or duplicate columns
        self.core = self.merge_columns(self.core)


        

         

        # Save the unsorted DataFrame to CSV first
        self.core.to_csv(self.output_csv_path, index=False)

        # Sort by 'q_id' after saving and then reload the file to avoid issues with sorting
        sorted_data = self.core.sort_values(by="q_id", ascending=True, na_position='last')
        sorted_data.to_csv(self.output_csv_path, index=False)
        print(f"Data extracted and saved to {self.output_csv_path}")
        print(f"Total PDFs accessed: {self.pdf_count}")
        print(f"Total PDFs successfully scraped: {self.scraped_count}")
 '''
    @staticmethod
    def find_table_title(page_text, table_identifier):
        table_title_match = re.search(rf"({table_identifier}[^\n]+)", page_text)
        return table_title_match.group(1) if table_title_match else None

    @staticmethod
    def search_in_table_title(table_title, patterns):
        for pattern in patterns:
            if re.search(pattern, table_title, re.IGNORECASE):
                return pattern
        return ""

# Usage
directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/matched_pdfs_phase_i/"
output_csv_path = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/c7_data.csv"
cluster7 = Cluster7(directory, output_csv_path)
cluster7.process_pdfs_in_folder()
cluster7.save_to_csv()


Processing C7PhI PG&E Fresno - Appendix I- Deliverability Assessment Results - Fresno.pdf
Skipping C7PhI PG&E Fresno - Appendix I- Deliverability Assessment Results - Fresno.pdf due to an error: No objects to concatenate
Processing C7 PhI PG&E Kern - Appendix I- Deliverability Assessment Results.pdf
Skipping C7 PhI PG&E Kern - Appendix I- Deliverability Assessment Results.pdf due to an error: No objects to concatenate
Processing C7 PhI PG&E North - Appendix I- Deliverability Assessment Results.pdf
Skipping C7 PhI PG&E North - Appendix I- Deliverability Assessment Results.pdf due to an error: No objects to concatenate
Processing C7PhI Appendix A - Q1037 New HECA_Final.pdf
Scraped data from C7PhI Appendix A - Q1037 New HECA_Final.pdf
Processing C7PhI - Appendix A - Q1030   South Lake Solar.pdf
Scraped data from C7PhI - Appendix A - Q1030   South Lake Solar.pdf
Processing QC7P1-SCE-VEA-EOP-Q1064-ARES-Appendix A.pdf
Skipping QC7P1-SCE-VEA-EOP-Q1064-ARES-Appendix A.pdf due to an error: No o

Process SpawnProcess-28:
Process SpawnProcess-26:
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=True)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'process_project' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "/Libr

Total number of files checked: 0
Total number of files moved to '02_phase_1_study': 0


In [9]:
import pandas as pd
import re
# Load the CSV file
df = pd.read_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/c7_original.csv')

def convert_to_snake_case(column_name):
    # Remove leading/trailing whitespaces and convert to lowercase
    column_name = column_name.strip().lower()
    # Replace spaces, hyphens, and other delimiters with underscores
    column_name = re.sub(r'[\s\-]+', '_', column_name)
    # Remove any non-alphanumeric characters except for underscores
    column_name = re.sub(r'[^\w]', '', column_name)
    return column_name





# Apply the function to all columns
df.columns = [convert_to_snake_case(col) for col in df.columns]

#df.rename(columns={'type of upgrade': 'type_of_upgrade'}, inplace=True)
#df.rename(columns={'cost allocation factor': 'cost_allocation_factor'}, inplace=True)



df['item'] = df.apply(
    lambda row: 'no' if (pd.notna(row['type_of_upgrade']) and 'Total' in row['type_of_upgrade']) or 
                         (pd.notna(row['cost_allocation_factor']) and 'Total' in row['cost_allocation_factor'])
               else 'yes', axis=1)

# 2. Move 'item' column next to 'type_of_upgrade' column
cols = df.columns.tolist()
item_index = cols.index('item')
cols.insert(cols.index('type_of_upgrade') + 1, cols.pop(item_index))
df = df[cols]

# 3. Remove the 'Total' values from the 'cost_allocation_factor' column if they are already in the 'type_of_upgrade' column
df['cost_allocation_factor'] = df.apply(
    lambda row: None if pd.notna(row['type_of_upgrade']) and 'Total' in row['type_of_upgrade'] else row['cost_allocation_factor'], axis=1
)

# Step 4: For each `q_id` and `type_of_upgrade`, if it has only one row and no 'Total' is present in 'cost_allocation_factor', create a new `Total` row
new_rows = []
for q_id, group in df.groupby('q_id'):
    unique_upgrades = group['type_of_upgrade'].unique()

    # Check if there's already a "Total" in 'cost_allocation_factor' for this q_id
    if any('Total' in str(x) for x in group['cost_allocation_factor']):
        continue  # If Total exists under 'cost_allocation_factor', skip creating new total row for this q_id
    
    for upgrade in unique_upgrades:
        # Skip if "Total" is already present for this upgrade or if the upgrade is NaN
        if pd.isna(upgrade) or 'Total' in str(upgrade):
            continue
        
        # Get rows corresponding to this specific upgrade
        rows = group[group['type_of_upgrade'] == upgrade]
        
        if len(rows) == 1:
            # Step 4.1: Duplicate the row and create a new row with "Total" under 'type_of_upgrade'
            original_row = rows.iloc[0].copy()
            total_row = original_row.copy()

            # Modify the total_row to reflect "Total"
            total_row['type_of_upgrade'] = 'Total'
            total_row['item'] = 'no'

            # Step 4.2: Append the Total row immediately after the original row
            original_index = df[(df['q_id'] == q_id) & (df['type_of_upgrade'] == upgrade)].index[0]
            new_rows.append((original_index + 1, total_row))

# Step 5: Insert the new rows into the DataFrame in the correct order
for idx, row in sorted(new_rows, reverse=True):
    df = pd.concat([df.iloc[:idx], pd.DataFrame([row]), df.iloc[idx:]]).reset_index(drop=True)
# Save the updated DataFrame to CSV

# Step 1: Move "Total" from 'cost allocation factor' column to 'type_of_upgrade' column
df['type_of_upgrade'] = df.apply(
    lambda row: 'Total' if 'Total' in str(row['cost_allocation_factor']) and 'Total' not in str(row['type_of_upgrade']) else row['type_of_upgrade'],
    axis=1
)

# Step 2: After moving "Total" to 'type_of_upgrade', clear it from 'cost_allocation_factor'
df['cost_allocation_factor'] = df.apply(
    lambda row: None if 'Total' in str(row['cost_allocation_factor']) else row['cost_allocation_factor'],
    axis=1
)

# Now, forward fill the non-empty values in 'type_of_upgrade' to replace empty cells with the previous value
df['type_of_upgrade'] = df['type_of_upgrade'].ffill()


previous_type_of_upgrade = None

for i in range(len(df)):
    # If the current row has 'Total' in 'type_of_upgrade' column
    if df.at[i, 'type_of_upgrade'] == 'Total':
        # Replace it with the previous valid 'type_of_upgrade' value
        if previous_type_of_upgrade is not None:
            df.at[i, 'type_of_upgrade'] = previous_type_of_upgrade
    else:
        # Update the previous_type_of_upgrade with the current row's value
        previous_type_of_upgrade = df.at[i, 'type_of_upgrade']
        

df = df.sort_values(by=['q_id', 'type_of_upgrade', 'upgrade'], ascending=[True, True, False])       

df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/c7_itemized.csv', index=False)





# Load the cleaned CSV file
cleaned_df = pd.read_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/c7_itemized.csv')  # Replace with the path to your cleaned CSV

# Filter the rows where 'item' is 'no'
total_rows_df = cleaned_df[cleaned_df['item'] == 'no']

# Drop the 'upgrade' and 'description' columns
total_rows_df = total_rows_df.drop(columns=['upgrade', 'description', 'cost_allocation_factor', 'estimated_time_to_construct', 'project_size'])

# Save the filtered DataFrame to a new CSV file
total_rows_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/c7_totals.csv', index=False)

print(f"Filtered Total rows saved to 'C7totals_only.csv'.")




Filtered Total rows saved to 'C7totals_only.csv'.


# cleaning

In [351]:
import pandas as pd
import re

# Load the CSV file, treating 'None', 'N/A', 'n/a' as missing values
missing_values = ['None', 'none', 'N/A', 'n/a', 'NaN']
df = pd.read_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/c7_original.csv', na_values=missing_values, keep_default_na=False)

# Function to convert column names to snake_case
def convert_to_snake_case(column_name):
    column_name = column_name.strip().lower()
    column_name = re.sub(r'[\s\-]+', '_', column_name)
    column_name = re.sub(r'[^\w]', '', column_name)
    return column_name






# Apply the function to all columns
df.columns = [convert_to_snake_case(col) for col in df.columns]

# List of numeric columns and non-numeric columns based on dataset structure
numeric_columns = ['cost_allocation_factor',  'estimated_cost_x_1000', 'estimated_time_to_construct', 'total_estimated_cost_x_1000_escalalted', 'project_size', 'adnu_cost_rate_x_1000', 'escalated_cost_x_1000', ]
non_numeric_columns = ['type_of_upgrade', 'upgrade', 'description']



# Step 2: Replace missing values in numeric columns with 0
df[numeric_columns] = df[numeric_columns].fillna(0)


# Step 3: Create the 'item' column based on whether the 'type_of_upgrade' row or 'cost_allocation_factor' contains 'Total'
df['item'] = df.apply(
    lambda row: 'no' if (pd.notna(row['type_of_upgrade']) and isinstance(row['type_of_upgrade'], str) and 'Total' in row['type_of_upgrade']) or 
                         (pd.notna(row['cost_allocation_factor']) and isinstance(row['cost_allocation_factor'], str) and 'Total' in row['cost_allocation_factor'])
               else 'yes', axis=1)

# Step 4: Move 'item' column next to 'type_of_upgrade' column
cols = df.columns.tolist()
item_index = cols.index('item')
cols.insert(cols.index('type_of_upgrade') + 1, cols.pop(item_index))
df = df[cols]

# Step 5: Remove the 'Total' values from the 'cost_allocation_factor' column if they are already in the 'type_of_upgrade' column
df['cost_allocation_factor'] = df.apply(
    lambda row: None if pd.notna(row['type_of_upgrade']) and 'Total' in row['type_of_upgrade'] else row['cost_allocation_factor'], axis=1
)

# Step 6: For each `q_id` and `type_of_upgrade`, if it has only one row and no 'Total' is present in 'cost_allocation_factor', create a new `Total` row
new_rows = []
for q_id, group in df.groupby('q_id'):
    unique_upgrades = group['type_of_upgrade'].unique()

    # Check if there's already a "Total" in 'cost_allocation_factor' for this q_id
    if any('Total' in str(x) for x in group['cost_allocation_factor']):
        continue  # Skip creating new total row for this q_id if Total exists under 'cost_allocation_factor'
    
    for upgrade in unique_upgrades:
        # Skip if "Total" is already present for this upgrade or if the upgrade is NaN
        if pd.isna(upgrade) or 'Total' in str(upgrade):
            continue
        
        # Get rows corresponding to this specific upgrade
        rows = group[group['type_of_upgrade'] == upgrade]
        
        if len(rows) == 1:
            # Duplicate the row and create a new row with "Total" under 'type_of_upgrade'
            original_row = rows.iloc[0].copy()
            total_row = original_row.copy()

            # Modify the total_row to reflect "Total"
            total_row['type_of_upgrade'] = 'Total'
            total_row['item'] = 'no'

            # Append the Total row immediately after the original row
            original_index = df[(df['q_id'] == q_id) & (df['type_of_upgrade'] == upgrade)].index[0]
            new_rows.append((original_index + 1, total_row))

# Insert the new rows into the DataFrame in the correct order
for idx, row in sorted(new_rows, reverse=True):
    df = pd.concat([df.iloc[:idx], pd.DataFrame([row]), df.iloc[idx:]]).reset_index(drop=True)

# Move "Total" from 'cost_allocation_factor' column to 'type_of_upgrade' column
df['type_of_upgrade'] = df.apply(
    lambda row: 'Total' if 'Total' in str(row['cost_allocation_factor']) and 'Total' not in str(row['type_of_upgrade']) else row['type_of_upgrade'],
    axis=1
)

# After moving "Total" to 'type_of_upgrade', clear it from 'cost_allocation_factor'
df['cost_allocation_factor'] = df.apply(
    lambda row: None if 'Total' in str(row['cost_allocation_factor']) else row['cost_allocation_factor'],
    axis=1
)

# Forward fill the non-empty values in 'type_of_upgrade' to replace empty cells with the previous value
df['type_of_upgrade'] = df['type_of_upgrade'].ffill()

# Ensure 'Total' is correctly replaced in 'type_of_upgrade' when present
previous_type_of_upgrade = None

for i in range(len(df)):
    # If the current row has 'Total' in 'type_of_upgrade' column
    if df.at[i, 'type_of_upgrade'] == 'Total':
        # Replace it with the previous valid 'type_of_upgrade' value
        if previous_type_of_upgrade is not None:
            df.at[i, 'type_of_upgrade'] = previous_type_of_upgrade
    else:
        # Update the previous_type_of_upgrade with the current row's value
        previous_type_of_upgrade = df.at[i, 'type_of_upgrade']

# Step 1: Replace missing values in non-numeric columns with None
df[non_numeric_columns] = df[non_numeric_columns].applymap(lambda x: None if pd.isna(x) else x)        

# Step 7: Sort the DataFrame by q_id and type_of_upgrade
df = df.sort_values(by=['q_id', 'type_of_upgrade', 'upgrade'], ascending=[True, True, False])

# Step 8: Create the itemized dataset (rows where item == 'yes') and save to CSV
itemized_df = df[df['item'] == 'yes']
itemized_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/c7_itemized.csv', index=False)

# Step 9: Create the totals dataset (rows where item == 'no') and drop unwanted columns
totals_df = df[df['item'] == 'no'].drop(columns=['upgrade', 'description', 'cost_allocation_factor', 'estimated_time_to_construct', 'project_size'])
totals_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/c7_totals.csv', index=False)

print(f"Itemized rows saved to 'c7_itemized.csv'.")
print(f"Filtered Total rows saved to 'c7_totals.csv'.")
 


Itemized rows saved to 'c7_itemized.csv'.
Filtered Total rows saved to 'c7_totals.csv'.


/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_88746/2653008972.py:108: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[non_numeric_columns] = df[non_numeric_columns].applymap(lambda x: None if pd.isna(x) else x)


# One more C7 try

In [59]:
import os
import pdfplumber
import pandas as pd
import re
import PyPDF2
from tabulate import tabulate

class Cluster7:
    def __init__(self, directory, output_csv_path):
        self.directory = directory
        self.output_csv_path = output_csv_path
        self.core = pd.DataFrame()  # Initialize an empty DataFrame to store all tables
        self.cluster_7_patterns = [
            r"C[\s_-]*7", r"c[\s_-]*7", r"Cluster[\s_-]*7", r"cluster[\s_-]*7", 
            r"C[\s_-]*VII", r"c[\s_-]*VII", r"Cluster[\s_-]*VII", r"cluster[\s_-]*VII"
        ]
        self.pdf_count = 0
        self.scraped_count = 0

    def clean_column_headers(self, headers):
        cleaned_headers = []
        for header in headers:
            if header is None:
                header = ""
            elif isinstance(header, str):
                header = header.lower()
                header = re.sub(r'\s+', ' ', header)
                header = re.sub(r'\(.*?\)', '', header)  # Remove anything in parentheses
                header = re.sub(r'[^a-zA-Z0-9\s]', '', header)
                header = header.strip()
            cleaned_headers.append(header)
        return cleaned_headers

    def clean_string_cell(self, value):
        if isinstance(value, str):
            return value.replace('\n', ' ').strip()  # Remove newlines and extra spaces
        return value

    def contains_phrase(self, row, phrase):
        regex_pattern = re.sub(r"\s+", r"\\s*", phrase)
        pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
        return row.astype(str).apply(lambda cell: bool(pattern.search(cell))).any()
    
    
    def merge_columns(self, df):
        """Merge columns that have similar names or convey the same information."""
        merge_columns = {
            "escalated cost x 1000": ["escalated costs x 1000", "estimated cost x 1000 escalated"],
            "estimated time to construct": ["estimated time to construct", "estimated time  to construct"],
            "total estimated cost x 1000 escalalted": ["total estimated cost x 1000 escalalted", "total estimated cost x 1000 escalated"],
            "adnu cost rate x 1000": ["adnu cost rate x 1000", "cost rate x 1000"],
            "description": ["description"],
            "capacity": ["capacity", "project size"],
        }
        

        # Identify unnamed columns (empty or NaN header)
        unnamed_columns = [col for col in df.columns if pd.isna(col) or col.strip() == ""]
        
        if unnamed_columns:
            # Merge unnamed columns into 'description' if they exist
            merge_columns["description"].extend(unnamed_columns)
        
        for new_col, old_cols in merge_columns.items():
            if any(col in df.columns for col in old_cols):
                df[new_col] = df[old_cols].bfill(axis=1).iloc[:, 0]
                df.drop(columns=[col for col in old_cols if col in df.columns and col != new_col], inplace=True)
        
        return df
      

    def extract_point_of_interconnection(self, pdf_path):
        """Extract the 'Point of Interconnection' from either Table 1 or using regex."""
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if "Table 1-" in page_text:
                    tables = page.extract_tables(table_settings={
                        "horizontal_strategy": "lines",
                        "vertical_strategy": "lines"
                    })
                    for table in tables:
                        for row in table:
                            if row and "Point of Interconnection" in row[0]:
                                return row[1]

        # Fallback to regex search in the text
        with open(pdf_path, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in reader.pages:
                text += page.extract_text()

        point_of_interconnection_match = re.search(r"point of interconnection[:\s]*(.+)", text, re.IGNORECASE)
        if point_of_interconnection_match:
            return point_of_interconnection_match.group(1).strip()

        return None

    def search_gps_coordinates(self, text):
        """Search for GPS coordinates using multiple patterns."""
        gps_coords = re.search(r"gps coordinates:\s*([\d\.\-]+),\s*([\d\.\-]+)", text, re.IGNORECASE)
        if gps_coords:
            return gps_coords.groups()

        project_coords = re.search(r"latitude[:\s]*([\d\.\-]+)[^\d]+longitude[:\s]*([\d\.\-]+)", text, re.IGNORECASE)
        if project_coords:
            return project_coords.groups()

        gps_coords_directional = re.search(
            r"gps coordinates:\s*([\d\.\-]+)\s*[nNsS],\s*([\d\.\-]+)\s*[eEwW]", text, re.IGNORECASE)
        if gps_coords_directional:
            lat, lon = gps_coords_directional.groups()
            latitude = lat if "N" in text.upper() else f"-{lat}"  # Adjust latitude sign
            longitude = lon if "E" in text.upper() else f"-{lon}"  # Adjust longitude sign
            return (latitude, longitude)

        return (None, None)

    def extract_pdf_data(self, pdf_path):
        """Extract base data from the PDF and return as a DataFrame."""
        with open(pdf_path, 'rb') as pdf:
            reader = PyPDF2.PdfReader(pdf)
            text = ""
            for page in reader.pages:
                text += page.extract_text()

        text = self.clean_string_cell(text)

        queue_id = re.search(r"q[\s_-]*(\d+)", text, re.IGNORECASE)
        queue_id = queue_id.group(1) if queue_id else None

        cluster_number = re.search(r"queue[\s_-]*cluster[\s_-]*(\d+)", text)
        cluster_number = cluster_number.group(1) if cluster_number else "7"

        deliverability_status = re.search(r"(\w+)\s*capacity deliverability status", text, re.IGNORECASE)
        deliverability_status = deliverability_status.group(1) if deliverability_status else None

        # Search for GPS coordinates
        latitude, longitude = self.search_gps_coordinates(text)

        capacity = re.search(r"total rated output of (\d+)\s*mw", text, re.IGNORECASE)
        if capacity:
            capacity = int(capacity.group(1))
        else:
            capacity2 = re.search(r"(\d+)\s*mw", text)
            capacity = int(capacity2.group(1)) if capacity2 else None

        # Extract Point of Interconnection
        point_of_interconnection = self.extract_point_of_interconnection(pdf_path)

        # Initialize base data dictionary
        base_data = {
            "q_id": [queue_id],
            "cluster": [cluster_number],
            "req_deliverability": [deliverability_status],
            "latitude": [latitude],
            "longitude": [longitude],
            "capacity": [capacity],
            "point_of_interconnection": [point_of_interconnection]
        }

        return pd.DataFrame(base_data)

    '''
    def extract_table7_and_replace_none(self, pdf_path):
        base_data = self.extract_pdf_data(pdf_path)
        pdf_core = pd.DataFrame()

        with pdfplumber.open(pdf_path) as pdf:
            table_in_progress = pd.DataFrame()
            headers = None
            expected_columns = None
            title_pattern = None
            last_known_upgrade = None 

            for page_number, page in enumerate(pdf.pages):
                text = page.extract_text()



                

                if re.search(r"Table 7", text):
                    tables = page.find_tables(table_settings={
                        "horizontal_strategy": "lines",
                        "vertical_strategy": "lines",
                    })

                    if not title_pattern:
                        table_title = self.find_table_title(text, "Table 7")
                        title_pattern = self.search_in_table_title(table_title, ["PTO", "Reliability Network Upgrade", "ADNU", "LDNU", "RNU"])

                    if not tables:
                        continue

                    for table_index, table in enumerate(tables):
                        tab = table.extract()
                        if len(tab) == 0:
                            continue

                        current_headers = tab[0]
                        cleaned_headers = self.clean_column_headers(current_headers)
                        data_rows = tab[1:]

                        replaced = False
                        for row in data_rows:
                            if any(cell and "Total" in cell for cell in row):
                               break
                            
                            if not replaced and (row[0] is None or row[0].strip() == ""):
                                row[0] = title_pattern
                                replaced = True

                        title_pattern = None        
                            
                    
                        

                            if row[0] is None or row[0].strip() == "":
                                # If no previous value, use the title_pattern as default
                                if last_known_upgrade is None:
                                    row[0] = title_pattern
                                else:
                                    row[0] = last_known_upgrade  # Use the last known non-None value
                            else:
                                # Update last_known_upgrade with the current non-None value
                                last_known_upgrade = row[0]
                          
                                


                                
                        if headers and len(current_headers) == expected_columns:
                            continuation_data = pd.DataFrame(tab, columns=headers)
                            table_in_progress = pd.concat([table_in_progress, continuation_data], ignore_index=True)
                        else:
                            if not table_in_progress.empty:
                                pdf_core = pd.concat([pdf_core, table_in_progress], ignore_index=True)
                                table_in_progress = pd.DataFrame()

                            headers = cleaned_headers
                            expected_columns = len(headers)
                            table_in_progress = pd.DataFrame(data_rows, columns=headers)
                             

            if not table_in_progress.empty:
                pdf_core = pd.concat([pdf_core, table_in_progress], ignore_index=True)

        overlapping_columns = base_data.columns.intersection(pdf_core.columns)
        pdf_core = pdf_core.drop(columns=overlapping_columns)
        merged_df = pd.concat([base_data] * len(pdf_core), ignore_index=True).join(pdf_core)

        return merged_df

    '''



    def extract_table7_and_replace_none(self, pdf_path):
        base_data = self.extract_pdf_data(pdf_path)
        pdf_core = pd.DataFrame()

        with pdfplumber.open(pdf_path) as pdf:
            table_in_progress = pd.DataFrame()
            headers = None
            expected_columns = None
            title_queue = []  # Queue to store titles that need tables extracted

            for page_number in range(len(pdf.pages)):
                page = pdf.pages[page_number]
                text = page.extract_text()
                if not text:
                    continue

                # Extract all instances of "Table 7" titles on the current page
                lines = text.split('\n')
                for line in lines:
                    if re.search(r"Table 7", line, re.IGNORECASE):
                        table_title = self.find_table_title(line, "Table 7")
                        title_pattern = self.search_in_table_title(table_title, ["PTO", "Reliability Network Upgrade", "ADNU", "LDNU", "RNU"])
                        title_queue.append(title_pattern)

                # Find all tables on the current page
                tables = page.find_tables(table_settings={
                    "horizontal_strategy": "lines",
                    "vertical_strategy": "lines",
                })

                # If there are no tables but a title is waiting, move to the next page
                if not tables and title_queue:
                    continue

                # Associate titles with tables on the same page or across pages
                for table_index, table in enumerate(tables):
                    if title_queue:
                        # Extract the current table and associate it with the current title in the queue
                        tab = table.extract()
                        if len(tab) > 0:
                            if headers and expected_columns == len(tab[0]):
                                # If headers match, this is a continuation of the previous table
                                data_rows = tab  # All rows are data rows in a continuation
                            else:
                                # New table with headers and data rows
                                current_headers = tab[0]
                                cleaned_headers = self.clean_column_headers(current_headers)
                                headers = cleaned_headers
                                expected_columns = len(headers)
                                data_rows = tab[1:]  # Skip the header row

                            # Assign the title to the table rows if missing
                            replaced = False
                            for row in data_rows:
                                if any(cell and "Total" in cell for cell in row):
                                    break

                                if not replaced and (row[0] is None or row[0].strip() == ""):
                                    row[0] = title_queue[0]  # Assign the current title in the queue to the table
                                    replaced = True

                            # Append rows to the ongoing table
                            continuation_data = pd.DataFrame(data_rows, columns=headers)
                            table_in_progress = pd.concat([table_in_progress, continuation_data], ignore_index=True)

                            # Remove the processed title from the queue
                            title_queue.pop(0)

                # If any titles remain in the queue without a corresponding table on the current page,
                # they will be processed on the next page.
                if title_queue and not tables:
                    continue

                # If there is still a table in progress at the end of the page, add it to the final DataFrame
                if not table_in_progress.empty:
                    pdf_core = pd.concat([pdf_core, table_in_progress], ignore_index=True)
                    table_in_progress = pd.DataFrame()

            # If there is still an unprocessed table at the end, add it to the final DataFrame
            if not table_in_progress.empty:
                pdf_core = pd.concat([pdf_core, table_in_progress], ignore_index=True)

        # Join the base data with the scraped table data
        overlapping_columns = base_data.columns.intersection(pdf_core.columns)
        pdf_core = pdf_core.drop(columns=overlapping_columns)
        merged_df = pd.concat([base_data] * len(pdf_core), ignore_index=True).join(pdf_core)

        return merged_df
  
         

    def process_pdfs_in_folder(self):
        for root, dirs, files in os.walk(self.directory):
            for pdf_name in files:
                if pdf_name.endswith(".pdf") and any(re.search(pattern, pdf_name, re.IGNORECASE) for pattern in self.cluster_7_patterns):
                    pdf_path = os.path.join(root, pdf_name)
                    self.pdf_count += 1
                    print(f"Processing {pdf_name}")

                    try:
                        df = self.extract_table7_and_replace_none(pdf_path)
                        if not df.empty:
                            self.core = pd.concat([self.core, df], ignore_index=True)
                            self.scraped_count += 1
                            print(f"Scraped data from {pdf_name}")
                    except Exception as e:
                        print(f"Skipping {pdf_name} due to an error: {e}")


    def save_to_csv(self):
        # Clean up the entire DataFrame by cleaning string cells
        self.core = self.core.map(self.clean_string_cell)

        # Drop rows that contain specific phrases (e.g., "Type of Upgrade")
        self.core = self.core[~self.core.apply(lambda row: self.contains_phrase(row, "Type of Upgrade"), axis=1)]

        # Merge similar or duplicate columns
        self.core = self.merge_columns(self.core)

        # Ensure q_id is numeric for sorting, replace missing values with None
        self.core['q_id'] = pd.to_numeric(self.core['q_id'], errors='coerce')

        # Save the unsorted DataFrame to CSV first
        self.core.to_csv(self.output_csv_path, index=False)

        # Sort by 'q_id' after saving and reload the file to avoid issues with sorting
        #sorted_data = self.core.sort_values(by="q_id", ascending=True, na_position='last')
        #sorted_data.to_csv(self.output_csv_path, index=False)

        print(f"Data extracted and saved to {self.output_csv_path}")
        print(f"Total PDFs accessed: {self.pdf_count}")
        print(f"Total PDFs successfully scraped: {self.scraped_count}")
                    
    '''  
    def save_to_csv(self):
        self.core = self.core.applymap(self.clean_string_cell)
        self.core = self.core[~self.core.apply(lambda row: self.contains_phrase(row, "Type of Upgrade"), axis=1)]
        # Sort by q_id in ascending order
        #self.core = self.core.sort_values(by="q_id", ascending=True)

        # Merge similar or duplicate columns
        self.core = self.merge_columns(self.core)


        

         

        # Save the unsorted DataFrame to CSV first
        self.core.to_csv(self.output_csv_path, index=False)

        # Sort by 'q_id' after saving and then reload the file to avoid issues with sorting
        sorted_data = self.core.sort_values(by="q_id", ascending=True, na_position='last')
        sorted_data.to_csv(self.output_csv_path, index=False)
        print(f"Data extracted and saved to {self.output_csv_path}")
        print(f"Total PDFs accessed: {self.pdf_count}")
        print(f"Total PDFs successfully scraped: {self.scraped_count}")
 '''
    @staticmethod
    def find_table_title(page_text, table_identifier):
        table_title_match = re.search(rf"({table_identifier}[^\n]+)", page_text)
        return table_title_match.group(1) if table_title_match else None

    @staticmethod
    def search_in_table_title(table_title, patterns):
        for pattern in patterns:
            if re.search(pattern, table_title, re.IGNORECASE):
                return pattern
        return ""

# Usage
directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/archive/matched_pdfs_phase_i/"
output_csv_path = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/c7_data.csv"
cluster7 = Cluster7(directory, output_csv_path)
cluster7.process_pdfs_in_folder()
cluster7.save_to_csv()


Processing C7PhI PG&E Fresno - Appendix I- Deliverability Assessment Results - Fresno.pdf
Skipping C7PhI PG&E Fresno - Appendix I- Deliverability Assessment Results - Fresno.pdf due to an error: No objects to concatenate
Processing C7 PhI PG&E Kern - Appendix I- Deliverability Assessment Results.pdf
Skipping C7 PhI PG&E Kern - Appendix I- Deliverability Assessment Results.pdf due to an error: No objects to concatenate
Processing C7 PhI PG&E North - Appendix I- Deliverability Assessment Results.pdf
Skipping C7 PhI PG&E North - Appendix I- Deliverability Assessment Results.pdf due to an error: No objects to concatenate
Processing C7PhI Appendix A - Q1037 New HECA_Final.pdf
Scraped data from C7PhI Appendix A - Q1037 New HECA_Final.pdf
Processing C7PhI - Appendix A - Q1030   South Lake Solar.pdf
Scraped data from C7PhI - Appendix A - Q1030   South Lake Solar.pdf
Processing QC7P1-SCE-VEA-EOP-Q1064-ARES-Appendix A.pdf
Skipping QC7P1-SCE-VEA-EOP-Q1064-ARES-Appendix A.pdf due to an error: No o

/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_8030/528258389.py:66: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[new_col] = df[old_cols].bfill(axis=1).iloc[:, 0]


# Cluster 6,7,8 old style

In [31]:
import os
import pdfplumber
import pandas as pd
import re
import PyPDF2
from tabulate import tabulate

class ClusterScraper:
    def __init__(self, base_directory, output_csv_path, project_range):
        self.base_directory = base_directory
        self.output_csv_path = output_csv_path
        self.project_range = project_range
        self.core = pd.DataFrame()  # Initialize an empty DataFrame to store all tables
        self.pdf_count = 0
        self.scraped_count = 0
        self.skipped_projects = []  # To store the list of skipped projects

    def clean_column_headers(self, headers):
        cleaned_headers = []
        for header in headers:
            if header is None:
                header = ""
            elif isinstance(header, str):
                header = header.lower()
                header = re.sub(r'\s+', ' ', header)
                header = re.sub(r'\(.*?\)', '', header)  # Remove anything in parentheses
                header = re.sub(r'[^a-zA-Z0-9\s]', '', header)
                header = header.strip()
            cleaned_headers.append(header)
        return cleaned_headers

    def clean_string_cell(self, value):
        if isinstance(value, str):
            return value.replace('\n', ' ').strip()  # Remove newlines and extra spaces
        return value

    def contains_phrase(self, row, phrase):
        regex_pattern = re.sub(r"\s+", r"\\s*", phrase)
        pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
        return row.astype(str).apply(lambda cell: bool(pattern.search(cell))).any()

    def merge_columns(self, df):
        """Merge columns that have similar names or convey the same information."""
        merge_columns = {
            "escalated cost x 1000": ["escalated costs x 1000", "estimated cost x 1000 escalated" , "estimated cost x 1000 escalated with itcca"],
            "estimated time to construct": ["estimated time to construct", "estimated time  to construct", "estimated time"],
            "total estimated cost x 1000 escalalted": ["total estimated cost x 1000 escalalted", "total estimated cost x 1000 escalated"],
            "adnu cost rate x 1000": ["adnu cost rate x 1000", "cost rate x 1000"],
            "description": ["description"],
            "capacity": ["capacity", "project size"],
            "cost allocation factor": ["cost allocation factor", "cost allocatio n factor"],


        }

        # Identify unnamed columns (empty or NaN header)
        unnamed_columns = [col for col in df.columns if pd.isna(col) or col.strip() == ""]
        
        if unnamed_columns:
            # Merge unnamed columns into 'description' if they exist
            merge_columns["description"].extend(unnamed_columns)
        
        for new_col, old_cols in merge_columns.items():
            if any(col in df.columns for col in old_cols):
                df[new_col] = df[old_cols].bfill(axis=1).iloc[:, 0]
                df.drop(columns=[col for col in old_cols if col in df.columns and col != new_col], inplace=True)
        
        return df
    

 

    def extract_point_of_interconnection(self, pdf_path):
        """Extract the 'Point of Interconnection' specifically from 'Table 1-' across multiple pages if necessary."""
        with pdfplumber.open(pdf_path) as pdf:
            table_in_progress = []
            poi_value = None
            extracting_table = False

            for page_number in range(len(pdf.pages)):
                page = pdf.pages[page_number]
                page_text = page.extract_text()

                # Start or continue extracting Table 1
                if "Table 1-" in page_text or extracting_table:
                    extracting_table = True  # Indicate we're in the process of extracting Table 1
                    tables = page.extract_tables(table_settings={
                        "horizontal_strategy": "lines",
                        "vertical_strategy": "lines"
                    })

                    for table in tables:
                        # If continuing a table from a previous page, extend the rows
                        table_in_progress.extend(table)

                        # Search through the rows to find "Point of Interconnection"
                        for row in table_in_progress:
                            if row and any(re.search(r"point of interconnection", str(cell), re.IGNORECASE) for cell in row):
                                # Extract the value corresponding to "Point of Interconnection"
                                poi_index = row.index(next(filter(lambda x: re.search(r"point of interconnection", str(x), re.IGNORECASE), row)))
                                if poi_index < len(row) - 1:
                                    # Extract the full content of the adjacent cell, even if it's multiline or has multiple parts
                                    poi_value = row[poi_index + 1]
                                    
                                    # If the value is a list, concatenate the strings
                                    if isinstance(poi_value, list):
                                        poi_value = " ".join(part.strip() for part in poi_value if isinstance(part, str))
                                    elif isinstance(poi_value, str):
                                        poi_value = poi_value.strip()

                                    # Return the fully cleaned Point of Interconnection value
                                    return poi_value

                    # If there's a need to continue the extraction to the next page
                    if not tables and extracting_table:
                        continue  # Proceed to the next page if the table continues

            return poi_value


    

    def search_gps_coordinates(self, text):
        """Search for GPS coordinates using multiple patterns."""
        gps_coords = re.search(r"gps coordinates:\s*([\d\.\-]+),\s*([\d\.\-]+)", text, re.IGNORECASE)
        if gps_coords:
            return gps_coords.groups()

        project_coords = re.search(r"latitude[:\s]*([\d\.\-]+)[^\d]+longitude[:\s]*([\d\.\-]+)", text, re.IGNORECASE)
        if project_coords:
            return project_coords.groups()

        gps_coords_directional = re.search(
            r"gps coordinates:\s*([\d\.\-]+)\s*[nNsS],\s*([\d\.\-]+)\s*[eEwW]", text, re.IGNORECASE)
        if gps_coords_directional:
            lat, lon = gps_coords_directional.groups()
            latitude = lat if "N" in text.upper() else f"-{lat}"  # Adjust latitude sign
            longitude = lon if "E" in text.upper() else f"-{lon}"  # Adjust longitude sign
            return (latitude, longitude)

        return (None, None)
    

    def extract_pdf_data(self, pdf_path, project_id):
        """Extract base data from the PDF and return as a DataFrame."""
        with open(pdf_path, 'rb') as pdf:
            reader = PyPDF2.PdfReader(pdf)
            text = ""
            for page in reader.pages:
                text += page.extract_text()

        text = self.clean_string_cell(text)

        queue_id = re.search(r"q[\s_-]*(\d+)", text, re.IGNORECASE)
        queue_id = queue_id.group(1) if queue_id else str(project_id)  # Use project_id if queue_id is not found

        cluster_number = re.search(r"queue[\s_-]*cluster[\s_-]*(\d+)", text, re.IGNORECASE)
        cluster_number = cluster_number.group(1)

        deliverability_status = re.search(r"(\w+)\s*capacity deliverability status", text, re.IGNORECASE)
        deliverability_status = deliverability_status.group(1) if deliverability_status else None

        # Search for GPS coordinates
        latitude, longitude = self.search_gps_coordinates(text)

        capacity = re.search(r"total rated output of (\d+)\s*mw", text, re.IGNORECASE)
        if capacity:
            capacity = int(capacity.group(1))
        else:
            capacity2 = re.search(r"(\d+)\s*mw", text)
            capacity = int(capacity2.group(1)) if capacity2 else None

        # Extract Point of Interconnection
        point_of_interconnection = self.extract_point_of_interconnection(pdf_path)

        # Initialize base data dictionary
        base_data = {
            "q_id": [queue_id],
            "cluster": [cluster_number],
            "req_deliverability": [deliverability_status],
            "latitude": [latitude],
            "longitude": [longitude],
            "capacity": [capacity],
            "point_of_interconnection": [point_of_interconnection]
        }

        return pd.DataFrame(base_data)

    def extract_table7_and_replace_none(self, pdf_path, project_id):
        #with pdfplumber.open(pdf_path) as pdf:
            #table7_found = any(re.search(r"Table 7", page.extract_text(), re.IGNORECASE) for page in pdf.pages if page.extract_text())
            #if not table7_found:
            #    print(f"Skipping {pdf_path} as no Table 7 found.")
             #   return pd.DataFrame()  # Return empty DataFrame if Table 7 is not found

        base_data = self.extract_pdf_data(pdf_path, project_id)
        pdf_core = pd.DataFrame()

        with pdfplumber.open(pdf_path) as pdf:
            table_in_progress = pd.DataFrame()
            headers = None
            expected_columns = None
            title_queue = []  # Queue to store titles that need tables extracted

            for page_number in range(len(pdf.pages)):
                page = pdf.pages[page_number]
                text = page.extract_text()
                if not text:
                    continue

                # Extract all instances of "Table 7" titles on the current page
                lines = text.split('\n')
                for line in lines:
                    if re.search(r"Table 7", line, re.IGNORECASE):
                        table_title = self.find_table_title(line, "Table 7")
                        title_pattern = self.search_in_table_title(table_title, ["PTO", "Reliability Network Upgrade", "ADNU", "LDNU", "RNU", "Area Delivery Network Upgrade", "Local Delivery Network"])
                        title_queue.append(title_pattern)

                # Find all tables on the current page
                tables = page.find_tables(table_settings={
                    "horizontal_strategy": "lines",
                    "vertical_strategy": "lines",
                })

                # If there are no tables but a title is waiting, move to the next page
                if not tables and title_queue:
                    continue

                # Associate titles with tables on the same page or across pages
                for table_index, table in enumerate(tables):
                    if title_queue:
                        # Extract the current table and associate it with the current title in the queue
                        tab = table.extract()
                        if len(tab) > 0:
                            if headers and expected_columns == len(tab[0]):
                                # If headers match, this is a continuation of the previous table
                                data_rows = tab  # All rows are data rows in a continuation
                            else:
                                # New table with headers and data rows
                                current_headers = tab[0]
                                cleaned_headers = self.clean_column_headers(current_headers)
                                headers = cleaned_headers
                                expected_columns = len(headers)
                                data_rows = tab[1:]  # Skip the header row

                            # Assign the title to the table rows if missing
                            replaced = False
                            for row in data_rows:
                                if any(cell and "Total" in cell for cell in row):
                                    break

                                if not replaced and (row[0] is None or row[0].strip() == ""):
                                    row[0] = title_queue[0]  # Assign the current title in the queue to the table
                                    replaced = True

                            # Append rows to the ongoing table
                            continuation_data = pd.DataFrame(data_rows, columns=headers)
                            table_in_progress = pd.concat([table_in_progress, continuation_data], ignore_index=True)

                            # Remove the processed title from the queue
                            title_queue.pop(0)

                            


                if title_queue and not tables:
                    continue
                
                # If there is still a table in progress at the end of the page, add it to the final DataFrame
                if not table_in_progress.empty:
                    pdf_core = pd.concat([pdf_core, table_in_progress], ignore_index=True)
                    table_in_progress = pd.DataFrame()

            # If there is still an unprocessed table at the end, add it to the final DataFrame
            if not table_in_progress.empty:
                pdf_core = pd.concat([pdf_core, table_in_progress], ignore_index=True)

        # Join the base data with the scraped table data
        #if not pdf_core.empty:
        overlapping_columns = base_data.columns.intersection(pdf_core.columns)
        pdf_core = pdf_core.drop(columns=overlapping_columns)
        merged_df = pd.concat([base_data] * len(pdf_core), ignore_index=True).join(pdf_core)
        #else:
        # If no valid tables were found, return base data only
        #   merged_df = base_data

        return merged_df

    def process_pdfs_in_folder(self):
        for project_id in self.project_range:
            project_path = os.path.join(self.base_directory, str(project_id), "02_phase_1_study")
            if not os.path.exists(project_path):
                continue

            self.pdf_count += 1
            for pdf_name in os.listdir(project_path):
                if pdf_name.endswith(".pdf"):
                    pdf_path = os.path.join(project_path, pdf_name)
                    print(f"Processing {pdf_name} from Project {project_id}")

                    try:
                        df = self.extract_table7_and_replace_none(pdf_path, project_id)
                        if not df.empty:
                            self.core = pd.concat([self.core, df], ignore_index=True)
                            self.scraped_count += 1
                            print(f"Scraped data from Project {project_id}, PDF: {pdf_name}")
                        else:
                            self.skipped_projects.append(project_id)
                    except Exception as e:
                        print(f"Skipping Project {project_id}, PDF: {pdf_name} due to an error: {e}")
                        self.skipped_projects.append(project_id)

        print(f"Projects skipped due to missing data: {self.skipped_projects}")


    def save_to_csv(self):
        # Clean up the entire DataFrame by cleaning string cells
        self.core = self.core.applymap(self.clean_string_cell)

        # Drop rows that contain specific phrases (e.g., "Type of Upgrade")
        self.core = self.core[~self.core.apply(lambda row: self.contains_phrase(row, "Type of Upgrade"), axis=1)]

        # Merge similar or duplicate columns
        #self.core = self.merge_columns(self.core)

        # Ensure q_id is numeric for sorting, replace missing values with None
        self.core['q_id'] = pd.to_numeric(self.core['q_id'], errors='coerce')

        # Save the DataFrame to CSV
        self.core.to_csv(self.output_csv_path, index=False)

        # Print final statistics
        print(f"Data extracted and saved to {self.output_csv_path}")
        print(f"Total project folders accessed: {self.pdf_count}")
        print(f"Total project folders successfully scraped: {self.scraped_count}")
        print(f"Projects skipped (no valid table structure): {self.skipped_projects}")

    @staticmethod
    def find_table_title(page_text, table_identifier):
        table_title_match = re.search(rf"({table_identifier}[^\n]+)", page_text)
        return table_title_match.group(1) if table_title_match else None

    @staticmethod
    def search_in_table_title(table_title, patterns):
        for pattern in patterns:
            if re.search(pattern, table_title, re.IGNORECASE):
                return pattern
        return ""

# Usage
base_directory = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data"
output_csv_path = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/C_6_7_8_data.csv"
project_range = range(943, 1220)  # Example range for q_ids in Clusters 6, 7, 8

cluster_scraper = ClusterScraper(base_directory, output_csv_path, project_range)
cluster_scraper.process_pdfs_in_folder()
cluster_scraper.save_to_csv()



Processing CAISO TC Phase I Appendix A East of Lugo.pdf from Project 154
Skipping Project 154, PDF: CAISO TC Phase I Appendix A East of Lugo.pdf due to an error: 'NoneType' object has no attribute 'group'
Processing CAISO TC Phase I Appendix A _East of Lugo_ 0731.pdf from Project 163
Skipping Project 163, PDF: CAISO TC Phase I Appendix A _East of Lugo_ 0731.pdf due to an error: 'NoneType' object has no attribute 'group'
Processing 07AS668411-Q163_Desert_Stateline_Project_Q163_GIDAPAppendix1AttachmentA.pdf from Project 163
Skipping Project 163, PDF: 07AS668411-Q163_Desert_Stateline_Project_Q163_GIDAPAppendix1AttachmentA.pdf due to an error: 'NoneType' object has no attribute 'group'
Processing TCPII_Q163_Appendix A Addendum.pdf from Project 163
Skipping Project 163, PDF: TCPII_Q163_Appendix A Addendum.pdf due to an error: 'NoneType' object has no attribute 'group'
Processing CAISO Q163 East of Lugo-Stateline PV 0731.pdf from Project 163
Skipping Project 163, PDF: CAISO Q163 East of Lugo

KeyboardInterrupt: 

# C 678 old style new code

In [1]:
import os
import pdfplumber
import pandas as pd
import re
import PyPDF2
from tabulate import tabulate

class ClusterScraper:
    def __init__(self, base_directory, output_csv_path, project_range):
        self.base_directory = base_directory
        self.output_csv_path = output_csv_path
        self.project_range = project_range
        self.core = pd.DataFrame()  # Initialize an empty DataFrame to store all tables
        self.pdf_count = 0
        self.scraped_count = 0
        self.skipped_projects = []  # To store the list of skipped projects

    def clean_column_headers(self, headers):
        cleaned_headers = []
        for header in headers:
            if header is None:
                header = ""
            elif isinstance(header, str):
                header = header.lower()
                header = re.sub(r'\s+', ' ', header)
                header = re.sub(r'\(.*?\)', '', header)  # Remove anything in parentheses
                header = re.sub(r'[^a-zA-Z0-9\s]', '', header)
                header = header.strip()
            cleaned_headers.append(header)
        return cleaned_headers

    def clean_string_cell(self, value):
        if isinstance(value, str):
            return value.replace('\n', ' ').strip()  # Remove newlines and extra spaces
        return value

    def contains_phrase(self, row, phrase):
        regex_pattern = re.sub(r"\s+", r"\\s*", phrase)
        pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
        return row.astype(str).apply(lambda cell: bool(pattern.search(cell))).any()

    def merge_columns(self, df):
        """Merge columns that have similar names or convey the same information."""
        merge_columns = {
            "escalated cost x 1000": ["escalated costs x 1000", "estimated cost x 1000 escalated", "estimated cost x 1000 escalated with itcca"],
            "estimated time to construct": ["estimated time to construct", "estimated time  to construct", "estimated time"],
            "total estimated cost x 1000 escalalted": ["total estimated cost x 1000 escalalted", "total estimated cost x 1000 escalated"],
            "adnu cost rate x 1000": ["adnu cost rate x 1000", "cost rate x 1000"],
            "description": ["description"],
            "capacity": ["capacity", "project size"],
            "cost allocation factor": ["cost allocation factor", "cost allocatio n factor"],
        }

        # Identify unnamed columns (empty or NaN header)
        unnamed_columns = [col for col in df.columns if pd.isna(col) or col.strip() == ""]

        if unnamed_columns:
            # Merge unnamed columns into 'description' if they exist
            merge_columns["description"].extend(unnamed_columns)

        for new_col, old_cols in merge_columns.items():
            if any(col in df.columns for col in old_cols):
                df[new_col] = df[old_cols].bfill(axis=1).iloc[:, 0]
                df.drop(columns=[col for col in old_cols if col in df.columns and col != new_col], inplace=True)

        return df

    def extract_point_of_interconnection(self, pdf_path):
        """Extract the 'Point of Interconnection' from 'Table 1-', handling multi-page tables."""
        #print("Extracting Point of Interconnection...")
        with pdfplumber.open(pdf_path) as pdf:
            table_in_progress = pd.DataFrame()
            headers = None
            expected_columns = None
            in_table1 = False

            for page_number in range(len(pdf.pages)):
                page = pdf.pages[page_number]
                text = page.extract_text()

                # Check if 'Table 1-' is in the text
                if re.search(r"Table\s*1", text, re.IGNORECASE) and not in_table1:
                    print(f"Found 'Table 1' on page {page_number + 1}")
                    in_table1 = True

                if in_table1:
                    print(f"Processing page {page_number + 1} for 'Table 1'...")
                    tables = page.extract_tables(table_settings={
                        "vertical_strategy": "lines",
                        "horizontal_strategy": "lines",
                    })

                    if not tables:
                        print(f"No tables found on page {page_number + 1}")
                        in_table1 = False
                        continue

                    for table_index, table in enumerate(tables):
                        tab = table

                        if len(tab) == 0:
                            continue

                        # Determine if this is a continuation table
                        if headers and len(tab[0]) == expected_columns:
                            # Continuation table
                            print(f"Detected continuation on page {page_number + 1}, table {table_index + 1}")
                            # Check if the first row matches the headers, skip if it does
                            first_row = tab[0]
                            cleaned_first_row = self.clean_column_headers(first_row)
                            if cleaned_first_row == headers:
                                data_rows = tab[1:]  # Skip the first row
                            else:
                                data_rows = tab  # All rows are data
                        else:
                            # New table
                            current_headers = tab[0]
                            cleaned_headers = self.clean_column_headers(current_headers)
                            headers = cleaned_headers
                            expected_columns = len(headers)
                            data_rows = tab[1:]  # Skip the header row
                            print(f"New table headers: {headers}")

                        # Create DataFrame for current data_rows
                        if headers and data_rows:
                            df_data = pd.DataFrame(data_rows, columns=headers)
                            table_in_progress = pd.concat([table_in_progress, df_data], ignore_index=True)
                        else:
                            print(f"Data row length mismatch on page {page_number + 1}, table {table_index + 1}")

                    # Check if the next page likely continues Table 1
                    if page_number + 1 < len(pdf.pages):
                        next_page = pdf.pages[page_number + 1]
                        next_text = next_page.extract_text()
                        if re.search(r"Table\s*\d", next_text, re.IGNORECASE) and not re.search(r"Table\s*1", next_text, re.IGNORECASE):
                            in_table1 = False
                            print(f"Found new table on page {page_number + 2}, ending Table 1 extraction.")
                    else:
                        in_table1 = False
                else:
                    continue

            if table_in_progress.empty:
                print("No data extracted from Table 1.")
                return None

            # Clean the DataFrame
            table_in_progress = table_in_progress.map(self.clean_string_cell)

            # Search for 'Point of Interconnection' in the DataFrame
            for idx, row in table_in_progress.iterrows():
                if row.astype(str).str.contains("point of interconnection", case=False, regex=False).any():
                    print(f"Found 'Point of Interconnection' in row {idx + 1}")
                    poi_index = row.index[row.astype(str).str.contains("point of interconnection", case=False, regex=False)][0]
                    poi_index_loc = row.index.get_loc(poi_index)
                    if poi_index_loc + 1 < len(row):
                        poi_value = row.iloc[poi_index_loc + 1]
                        #print(f"Extracted Point of Interconnection: {poi_value}")
                        return poi_value.strip()
                    else:
                        #print("Adjacent cell for 'Point of Interconnection' not found.")
                        return None

            #print("Point of Interconnection not found in Table 1.")
            return None

    def search_gps_coordinates(self, text):
        """Search for GPS coordinates using multiple patterns."""
       # print("Searching for GPS coordinates...")
        gps_coords = re.search(r"gps coordinates:\s*([\d\.\-]+),\s*([\d\.\-]+)", text, re.IGNORECASE)
        if gps_coords:
           # print(f"Found GPS coordinates: {gps_coords.groups()}")
            return gps_coords.groups()

        project_coords = re.search(r"latitude[:\s]*([\d\.\-]+)[^\d]+longitude[:\s]*([\d\.\-]+)", text, re.IGNORECASE)
        if project_coords:
            #print(f"Found project coordinates: {project_coords.groups()}")
            return project_coords.groups()

        gps_coords_directional = re.search(
            r"gps coordinates:\s*([\d\.\-]+)\s*[nNsS],\s*([\d\.\-]+)\s*[eEwW]", text, re.IGNORECASE)
        if gps_coords_directional:
            lat, lon = gps_coords_directional.groups()
            latitude = lat if "N" in text.upper() else f"-{lat}"  # Adjust latitude sign
            longitude = lon if "E" in text.upper() else f"-{lon}"  # Adjust longitude sign
            print(f"Found directional GPS coordinates: {(latitude, longitude)}")
            return (latitude, longitude)

        print("GPS coordinates not found.")
        return (None, None)

    def extract_pdf_data(self, pdf_path, project_id):
        """Extract base data from the PDF and return as a DataFrame."""
        print("Extracting base data from PDF...")
        with open(pdf_path, 'rb') as pdf:
            reader = PyPDF2.PdfReader(pdf)
            text = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text

        text = self.clean_string_cell(text)

        queue_id = re.search(r"q[\s_-]*(\d+)", text, re.IGNORECASE)
        queue_id = queue_id.group(1) if queue_id else str(project_id)  # Use project_id if queue_id is not found
        print(f"Extracted Queue ID: {queue_id}")

        cluster_number = re.search(r"queue[\s_-]*cluster[\s_-]*(\d+)", text, re.IGNORECASE)
        cluster_number = cluster_number.group(1) if cluster_number else None
        print(f"Extracted Cluster Number: {cluster_number}")

        deliverability_status = re.search(r"(\w+)\s*capacity deliverability status", text, re.IGNORECASE)
        deliverability_status = deliverability_status.group(1) if deliverability_status else None
        print(f"Extracted Deliverability Status: {deliverability_status}")

        # Search for GPS coordinates
        latitude, longitude = self.search_gps_coordinates(text)

        capacity = re.search(r"total rated output of (\d+)\s*mw", text, re.IGNORECASE)
        if capacity:
            capacity = int(capacity.group(1))
        else:
            capacity2 = re.search(r"(\d+)\s*mw", text)
            capacity = int(capacity2.group(1)) if capacity2 else None
        print(f"Extracted Capacity: {capacity}")

        # Extract Point of Interconnection
        point_of_interconnection = self.extract_point_of_interconnection(pdf_path)

        # Initialize base data dictionary
        base_data = {
            "q_id": [queue_id],
            "cluster": [cluster_number],
            "req_deliverability": [deliverability_status],
            "latitude": [latitude],
            "longitude": [longitude],
            "capacity": [capacity],
            "point_of_interconnection": [point_of_interconnection]
        }

        print("Base data extracted:", base_data)
        return pd.DataFrame(base_data)

    def extract_table7_and_replace_none(self, pdf_path, project_id):
        print("Extracting Table 7 and handling multi-page tables...")
        base_data = self.extract_pdf_data(pdf_path, project_id)
        pdf_core = pd.DataFrame()

        with pdfplumber.open(pdf_path) as pdf:
            table_in_progress = pd.DataFrame()
            headers = None
            expected_columns = None
            title_pattern = None
            in_table7 = False
            replaced = False  # Initialize 'replaced' here
            previous_title_pattern = None  # Keep track of previous table title

            for page_number, page in enumerate(pdf.pages):
                text = page.extract_text()

                # Check if "Table 7" is in the text
                if re.search(r"Table 7", text):
                    in_table7 = True
                    table_title = self.find_table_title(text, "Table 7")
                    title_pattern = self.search_in_table_title(table_title, ["PTO", "Reliability Network Upgrade", "Area Delivery Network Upgrade", "Local Delivery Network", "ADNU", "LDNU", "RNU"])
                    print(f"New title pattern detected: {title_pattern}")
                    previous_title_pattern = title_pattern  # Update previous title pattern
                    replaced = False  # Reset 'replaced' when starting a new table

                if in_table7:
                    # Extract all tables from the page
                    tables = page.find_tables(table_settings={
                        "horizontal_strategy": "lines",
                        "vertical_strategy": "lines",
                    })

                    if not tables:
                        print(f"No tables found on page {page_number + 1}")
                        in_table7 = False  # Assume table has ended
                        continue

                    for table_index, table in enumerate(tables):
                        tab = table.extract()

                        if len(tab) == 0:
                            continue

                        # Determine if this is a continuation table
                        if headers and len(tab[0]) == expected_columns:
                            # Continuation table
                            print(f"Detected continuation on page {page_number + 1}, table {table_index + 1}")
                            # Check if the first row matches the headers, skip if it does
                            first_row = tab[0]
                            cleaned_first_row = self.clean_column_headers(first_row)
                            if cleaned_first_row == headers:
                                data_rows = tab[1:]  # Skip the first row

                                if data_rows and not replaced:
                                    for idx, row in enumerate(data_rows):
                                        if row[0] is None or row[0].strip() == "":
                                            print(f"Setting 'Type of Upgrade' in data row: {row}")
                                            row[0] = previous_title_pattern  # Use previous title pattern
                                            replaced = True
                                            break  # Only replace once per table
                            else:
                                data_rows = tab  # All rows are data
                        else:
                            # New table
                            if not table_in_progress.empty:
                                print(f"Appending previous table to pdf_core from page {page_number + 1}")
                                pdf_core = pd.concat([pdf_core, table_in_progress], ignore_index=True)
                                table_in_progress = pd.DataFrame()
                            current_headers = tab[0]
                            cleaned_headers = self.clean_column_headers(current_headers)
                            data_rows = tab[1:]
                            headers = cleaned_headers
                            expected_columns = len(headers)
                            print(f"New table detected on page {page_number + 1}, table {table_index + 1} with {expected_columns} columns")
                            replaced = False  # Reset 'replaced' when starting a new table

                        # Create DataFrame for current data_rows
                        if headers and data_rows and len(data_rows[0]) == expected_columns:
                            df_data = pd.DataFrame(data_rows, columns=headers)
                            table_in_progress = pd.concat([table_in_progress, df_data], ignore_index=True)
                        else:
                            print(f"Data row length mismatch on page {page_number + 1}, table {table_index + 1}")
                else:
                    continue

            # Append any remaining table in progress to pdf_core after the last page
            if not table_in_progress.empty:
                print(f"Appending final table in progress to pdf_core for {pdf_path}")
                pdf_core = pd.concat([pdf_core, table_in_progress], ignore_index=True)

        # Clean up the entire DataFrame
        pdf_core = pdf_core.map(self.clean_string_cell)

        # Drop rows that contain specific phrases
        pdf_core = pdf_core[~pdf_core.apply(lambda row: self.contains_phrase(row, "Type of Upgrade"), axis=1)]

        # Merge base data with pdf_core
        if not pdf_core.empty:
            overlapping_columns = base_data.columns.intersection(pdf_core.columns)
            pdf_core = pdf_core.drop(columns=overlapping_columns)
            base_data_repeated = pd.concat([base_data] * len(pdf_core), ignore_index=True)
            merged_df = pd.concat([base_data_repeated, pdf_core], axis=1)
            print(f"Merged base data with table data for {pdf_path}")
        else:
            # If no table data extracted, use base data only
            merged_df = base_data
            print(f"No Table 7 data found in {pdf_path}. Using base data only.")

        return merged_df

    def process_pdfs_in_folder(self):
        for project_id in self.project_range:
            project_path = os.path.join(self.base_directory, str(project_id), "02_phase_1_study")
            if not os.path.exists(project_path):
                continue

            self.pdf_count += 1
            for pdf_name in os.listdir(project_path):
                if pdf_name.endswith(".pdf"):
                    pdf_path = os.path.join(project_path, pdf_name)
                    print(f"\nProcessing {pdf_name} from Project {project_id}")

                    try:
                        df = self.extract_table7_and_replace_none(pdf_path, project_id)
                        if not df.empty:
                            self.core = pd.concat([self.core, df], ignore_index=True)
                            self.scraped_count += 1
                            print(f"Scraped data from Project {project_id}, PDF: {pdf_name}")
                        else:
                            self.skipped_projects.append(project_id)
                    except Exception as e:
                        print(f"Skipping Project {project_id}, PDF: {pdf_name} due to an error: {e}")
                        self.skipped_projects.append(project_id)

        if self.skipped_projects:
            print(f"Projects skipped due to missing data: {self.skipped_projects}")

    def save_to_csv(self):
        # Clean up the entire DataFrame by cleaning string cells
        self.core = self.core.map(self.clean_string_cell)

        # Drop rows that contain specific phrases (e.g., "Type of Upgrade")
        self.core = self.core[~self.core.apply(lambda row: self.contains_phrase(row, "Type of Upgrade"), axis=1)]

        # Merge similar or duplicate columns
        #self.core = self.merge_columns(self.core)

        # Ensure q_id is numeric for sorting, replace missing values with None
        self.core['q_id'] = pd.to_numeric(self.core['q_id'], errors='coerce')

        # Save the DataFrame to CSV
        self.core.to_csv(self.output_csv_path, index=False)

        # Print final statistics
        print(f"Data extracted and saved to {self.output_csv_path}")
        print(f"Total project folders accessed: {self.pdf_count}")
        print(f"Total project folders successfully scraped: {self.scraped_count}")
        if self.skipped_projects:
            print(f"Projects skipped (no valid table structure): {self.skipped_projects}")

    @staticmethod
    def find_table_title(page_text, table_identifier):
        table_title_match = re.search(rf"({table_identifier}[^\n]*)", page_text)
        return table_title_match.group(1) if table_title_match else None

    @staticmethod
    def search_in_table_title(table_title, patterns):
        for pattern in patterns:
            if re.search(pattern, table_title, re.IGNORECASE):
                return pattern
        return ""

# Usage
base_directory = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data"
output_csv_path = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/C_6_7_8_data.csv"
project_range = range(943, 1220)  # Example range for q_ids in Clusters 6, 7, 8

cluster_scraper = ClusterScraper(base_directory, output_csv_path, project_range)
cluster_scraper.process_pdfs_in_folder()
cluster_scraper.save_to_csv()



Processing Q943 Lerdo Solar_C6Ph I_Appendix A-Individual Study Report_Final.pdf from Project 943
Extracting Table 7 and handling multi-page tables...
Extracting base data from PDF...
Extracted Queue ID: 943
Extracted Cluster Number: 6
Extracted Deliverability Status: Full
Found directional GPS coordinates: ('35.499773', '119.161457')
Extracted Capacity: 50
Found 'Table 1' on page 4
Processing page 4 for 'Table 1'...
No tables found on page 4
Found 'Table 1' on page 5
Processing page 5 for 'Table 1'...
New table headers: ['project parameters', 'project specific data']
Detected continuation on page 5, table 2
Processing page 6 for 'Table 1'...
No tables found on page 6
Found 'Point of Interconnection' in row 11
Base data extracted: {'q_id': ['943'], 'cluster': ['6'], 'req_deliverability': ['Full'], 'latitude': ['35.499773'], 'longitude': ['119.161457'], 'capacity': [50], 'point_of_interconnection': ['Lerdo 115 kV Substation']}
New title pattern detected: 
New table detected on page 12, 

In [44]:
import os
import pdfplumber
import pandas as pd
import re
import PyPDF2
from tabulate import tabulate

class PDFTableExtractor:
    def __init__(self):
        pass

    def clean_column_headers(self, headers):
        cleaned_headers = []
        for header in headers:
            if header is None:
                header = ""
            elif isinstance(header, str):
                header = header.lower()
                header = re.sub(r'\s+', ' ', header)
                header = re.sub(r'\(.*?\)', '', header)  # Remove anything in parentheses
                header = re.sub(r'[^a-zA-Z0-9\s]', '', header)
                header = header.strip()
            cleaned_headers.append(header)
        return cleaned_headers

    def clean_string_cell(self, value):
        if isinstance(value, str):
            return value.replace('\n', ' ').strip()  # Remove newlines and extra spaces
        return value

    def contains_phrase(self, row, phrase):
        regex_pattern = re.sub(r"\s+", r"\\s*", phrase)
        pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
        return row.astype(str).apply(lambda cell: bool(pattern.search(cell))).any()

    def search_gps_coordinates(self, text):
        """Search for GPS coordinates using multiple patterns."""
        gps_coords = re.search(r"gps coordinates:\s*([\d\.\-]+),\s*([\d\.\-]+)", text, re.IGNORECASE)
        if gps_coords:
            return gps_coords.groups()

        project_coords = re.search(r"latitude[:\s]*([\d\.\-]+)[^\d]+longitude[:\s]*([\d\.\-]+)", text, re.IGNORECASE)
        if project_coords:
            return project_coords.groups()

        gps_coords_directional = re.search(
            r"gps coordinates:\s*([\d\.\-]+)\s*[nNsS],\s*([\d\.\-]+)\s*[eEwW]", text, re.IGNORECASE)
        if gps_coords_directional:
            lat, lon = gps_coords_directional.groups()
            latitude = lat if "N" in text.upper() else f"-{lat}"  # Adjust latitude sign
            longitude = lon if "E" in text.upper() else f"-{lon}"  # Adjust longitude sign
            return (latitude, longitude)

        return (None, None)

    def extract_pdf_data(self, pdf_path, project_id):
        """Extract base data from the PDF and return as a DataFrame."""
        with open(pdf_path, 'rb') as pdf:
            reader = PyPDF2.PdfReader(pdf)
            text = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text

        text = self.clean_string_cell(text)

        queue_id = re.search(r"q[\s_-]*(\d+)", text, re.IGNORECASE)
        queue_id = queue_id.group(1) if queue_id else str(project_id)  # Use project_id if queue_id is not found

        cluster_number = re.search(r"queue[\s_-]*cluster[\s_-]*(\d+)", text, re.IGNORECASE)
        cluster_number = cluster_number.group(1) if cluster_number else None

        deliverability_status = re.search(r"(\w+)\s*capacity deliverability status", text, re.IGNORECASE)
        deliverability_status = deliverability_status.group(1) if deliverability_status else None

        # Search for GPS coordinates
        latitude, longitude = self.search_gps_coordinates(text)

        capacity = re.search(r"total rated output of (\d+)\s*mw", text, re.IGNORECASE)
        if capacity:
            capacity = int(capacity.group(1))
        else:
            capacity2 = re.search(r"(\d+)\s*mw", text)
            capacity = int(capacity2.group(1)) if capacity2 else None

        # Since we don't need data from Table 1, we won't call extract_point_of_interconnection

        # Initialize base data dictionary
        base_data = {
            "q_id": [queue_id],
            "cluster": [cluster_number],
            "req_deliverability": [deliverability_status],
            "latitude": [latitude],
            "longitude": [longitude],
            "capacity": [capacity],
            # "point_of_interconnection": [point_of_interconnection]  # Removed since we're not extracting it
        }

        return pd.DataFrame(base_data)

    def extract_table7_and_replace_none(self, pdf_path, project_id):
        """Extract Table 7 data from the PDF, handling tables split across pages."""
        base_data = self.extract_pdf_data(pdf_path, project_id)
        pdf_core = pd.DataFrame()

        with pdfplumber.open(pdf_path) as pdf:
            current_title = None
            current_title_pattern = None
            current_headers = None
            current_table_data = []
            extracting_table = False
            in_table7 = False  # Flag to indicate if we are within Table 7

            for page_number in range(len(pdf.pages)):
                page = pdf.pages[page_number]
                text = page.extract_text()
                if not text:
                    continue

                # Check if 'Table 7' is on this page
                if 'Table 7' in text:
                    in_table7 = True
                else:
                    # If 'Table 7' is not on this page, and we're not in a continuation, skip processing
                    if not extracting_table:
                        continue

                # Extract all instances of "Table 7" titles on the current page
                lines = text.split('\n')
                new_titles = []
                for line in lines:
                    if re.search(r"Table 7", line, re.IGNORECASE):
                        table_title = self.find_table_title(line, "Table 7")
                        title_pattern = self.search_in_table_title(table_title, ["PTO", "Reliability Network Upgrade", "ADNU", "LDNU", "RNU", "Area Delivery Network Upgrade", "Local Delivery Network"])
                        new_titles.append((table_title, title_pattern))
                        # We are now within Table 7
                        in_table7 = True

                # If new titles are found, handle existing data
                if new_titles:
                    if extracting_table and current_table_data:
                        # Save the current table data
                        print(f"\nProcessing table with title: {current_title}")
                        print(f"Current headers: {current_headers}")
                        print(f"Number of data rows: {len(current_table_data)}")
                        try:
                            df_table = pd.DataFrame(current_table_data, columns=current_headers)
                            print("DataFrame created successfully")
                            print("DataFrame before reset_index:")
                            print(df_table.head())
                            df_table.reset_index(drop=True, inplace=True)
                            print("DataFrame after reset_index:")
                            print(df_table.head())
                            # Replace None only in the first row of 'type of upgrade' with title_pattern
                            if not df_table.empty:
                                if 'type of upgrade' in df_table.columns:
                                    if pd.isnull(df_table.loc[0, 'type of upgrade']) or df_table.loc[0, 'type of upgrade'].strip() == '':
                                        df_table.loc[0, 'type of upgrade'] = current_title_pattern
                                else:
                                    df_table.insert(0, 'type of upgrade', [current_title_pattern] + [None]*(len(df_table)-1))
                                print("DataFrame after inserting 'type of upgrade':")
                                print(df_table.head())
                                print("Current pdf_core before concatenation:")
                                print(pdf_core.head())
                                pdf_core = pd.concat([pdf_core, df_table], ignore_index=True)
                                print("pdf_core after concatenation:")
                                print(pdf_core.head())
                        except Exception as e:
                            print(f"Error creating DataFrame for table starting with title '{current_title}': {e}")
                            print(f"current_headers: {current_headers}")
                            print(f"current_table_data: {current_table_data}")
                        current_table_data = []
                        current_headers = None
                    elif extracting_table and not current_table_data:
                        # No data to save, reset variables
                        current_headers = None

                    # Start new table with the first title found
                    current_title, current_title_pattern = new_titles[0]  # Use the full table title and pattern
                    extracting_table = True

                # Process tables only if we are within Table 7
                if in_table7:
                    # Find all tables on the current page
                    tables = page.find_tables(table_settings={
                        "horizontal_strategy": "lines",
                        "vertical_strategy": "lines",
                    })

                    if tables:
                        for table in tables:
                            tab = table.extract()
                            if len(tab) > 0:
                                # Determine if this is a continuation or a new table
                                if current_headers and len(tab[0]) == len(current_headers):
                                    # Continuation of previous table
                                    data_rows = tab  # All rows are data rows in a continuation
                                    print(f"Continuing table with headers: {current_headers}")
                                else:
                                    # Before starting a new table, check if current_table_data is not empty
                                    if current_table_data:
                                        # Save the current table data
                                        print(f"\nProcessing table with title: {current_title}")
                                        print(f"Current headers: {current_headers}")
                                        print(f"Number of data rows: {len(current_table_data)}")
                                        try:
                                            df_table = pd.DataFrame(current_table_data, columns=current_headers)
                                            print("DataFrame created successfully")
                                            print("DataFrame before reset_index:")
                                            print(df_table.head())
                                            df_table.reset_index(drop=True, inplace=True)
                                            print("DataFrame after reset_index:")
                                            print(df_table.head())
                                            # Replace None only in the first row of 'type of upgrade' with title_pattern
                                            if not df_table.empty:
                                                if 'type of upgrade' in df_table.columns:
                                                    if pd.isnull(df_table.loc[0, 'type of upgrade']) or df_table.loc[0, 'type of upgrade'].strip() == '':
                                                        df_table.loc[0, 'type of upgrade'] = current_title_pattern
                                                else:
                                                    df_table.insert(0, 'type of upgrade', [current_title_pattern] + [None]*(len(df_table)-1))
                                                print("DataFrame after inserting 'type of upgrade':")
                                                print(df_table.head())
                                                print("Current pdf_core before concatenation:")
                                                print(pdf_core.head())
                                                pdf_core = pd.concat([pdf_core, df_table], ignore_index=True)
                                                print("pdf_core after concatenation:")
                                                print(pdf_core.head())
                                        except Exception as e:
                                            print(f"Error creating DataFrame for table starting with title '{current_title}': {e}")
                                            print(f"current_headers: {current_headers}")
                                            print(f"current_table_data: {current_table_data}")
                                        current_table_data = []
                                    # Start new table
                                    current_headers = self.clean_column_headers(tab[0])
                                    data_rows = tab[1:]  # Skip the header row
                                    # Update current_title and current_title_pattern if there's no new title
                                    if not new_titles:
                                        current_title = None
                                        current_title_pattern = None
                                    print(f"Starting new table with headers: {current_headers}")

                                current_table_data.extend(data_rows)

                    else:
                        # Handle continuation tables without headers
                        if extracting_table and current_table_data:
                            # Assuming continuation tables have same number of columns
                            tab = page.extract_table(table_settings={
                                "horizontal_strategy": "lines",
                                "vertical_strategy": "lines",
                            })
                            if tab and len(tab[0]) == len(current_headers):
                                data_rows = tab  # All rows are data rows in a continuation
                                current_table_data.extend(data_rows)

                # If 'Table 7' is not on this page and we are not extracting a continuation, set in_table7 to False
                else:
                    in_table7 = False

            # At the end of the document, save any remaining data
            if extracting_table and current_table_data:
                if current_table_data:
                    print(f"\nProcessing final table with title: {current_title}")
                    print(f"Current headers: {current_headers}")
                    print(f"Number of data rows: {len(current_table_data)}")
                    try:
                        df_table = pd.DataFrame(current_table_data, columns=current_headers)
                        print("DataFrame created successfully")
                        print("DataFrame before reset_index:")
                        print(df_table.head())
                        df_table.reset_index(drop=True, inplace=True)
                        print("DataFrame after reset_index:")
                        print(df_table.head())
                        # Replace None only in the first row of 'type of upgrade' with title_pattern
                        if not df_table.empty:
                            if 'type of upgrade' in df_table.columns:
                                if pd.isnull(df_table.loc[0, 'type of upgrade']) or df_table.loc[0, 'type of upgrade'].strip() == '':
                                    df_table.loc[0, 'type of upgrade'] = current_title_pattern
                            else:
                                df_table.insert(0, 'type of upgrade', [current_title_pattern] + [None]*(len(df_table)-1))
                            print("DataFrame after inserting 'type of upgrade':")
                            print(df_table.head())
                            print("Current pdf_core before concatenation:")
                            print(pdf_core.head())
                            pdf_core = pd.concat([pdf_core, df_table], ignore_index=True)
                            print("pdf_core after concatenation:")
                            print(pdf_core.head())
                    except Exception as e:
                        print(f"Error creating DataFrame for table starting with title '{current_title}': {e}")
                        print(f"current_headers: {current_headers}")
                        print(f"current_table_data: {current_table_data}")

        # Join the base data with the scraped table data
        if not pdf_core.empty:
            overlapping_columns = base_data.columns.intersection(pdf_core.columns)
            pdf_core = pdf_core.drop(columns=overlapping_columns)
            try:
                print("\nMerging base_data and pdf_core")
                print("base_data:")
                print(base_data.head())
                print("pdf_core:")
                print(pdf_core.head())
                merged_df = pd.concat([base_data] * len(pdf_core), ignore_index=True).join(pdf_core.reset_index(drop=True))
                print("merged_df:")
                print(merged_df.head())
            except Exception as e:
                print(f"Error merging base_data and pdf_core: {e}")
        else:
            # If no valid tables were found, return base data only
            merged_df = base_data

        return merged_df

    @staticmethod
    def find_table_title(line, table_identifier):
        table_title_match = re.search(rf"({table_identifier}[^\n]*)", line)
        return table_title_match.group(1).strip() if table_title_match else None

    @staticmethod
    def search_in_table_title(table_title, patterns):
        for pattern in patterns:
            if re.search(pattern, table_title, re.IGNORECASE):
                return pattern
        return ""

    def process_pdfs(self, pdf_list):
        for pdf_path in pdf_list:
            if os.path.exists(pdf_path) and pdf_path.endswith(".pdf"):
                try:
                    print(f"\nProcessing {pdf_path}")
                    # Extract project_id from pdf_path if needed
                    project_id = os.path.basename(pdf_path).split('.')[0]  # Modify as appropriate
                    df = self.extract_table7_and_replace_none(pdf_path, project_id)
                    if not df.empty:
                        # Clean up the DataFrame by cleaning string cells
                        df = df.applymap(self.clean_string_cell)
                        # Optionally drop rows containing specific phrases
                        df = df[~df.apply(lambda row: self.contains_phrase(row, "Type of Upgrade"), axis=1)]
                        # Print the output using tabulate
                        print("\nFinal extracted data:")
                        print(tabulate(df, headers='keys', tablefmt='fancy_grid'))
                    else:
                        print(f"No data extracted from {pdf_path}")
                except Exception as e:
                    print(f"Error processing {pdf_path}: {e}")
            else:
                print(f"File not found or invalid: {pdf_path}")

# Usage example
pdf_list = [
    "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1030/02_phase_1_study/C7PhI - Appendix A - Q1030   South Lake Solar.pdf",
   "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1158/02_phase_1_study/Q1158 Slate_Appendix A Report_C8PhI.pdf",
    # Add more PDF paths as needed
]

extractor = PDFTableExtractor()
extractor.process_pdfs(pdf_list)



Processing /Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/1030/02_phase_1_study/C7PhI - Appendix A - Q1030   South Lake Solar.pdf
Starting new table with headers: ['type of upgrade', 'upgrade', 'description', 'cost allocation factor', 'estimated cost x 1000', 'escalated costs x 1000', 'estimated time  to construct']

Processing table with title: Table 7-1: Escalated Cost and Time to Construct for Interconnection Facilities - IF
Current headers: ['type of upgrade', 'upgrade', 'description', 'cost allocation factor', 'estimated cost x 1000', 'escalated costs x 1000', 'estimated time  to construct']
Number of data rows: 5
DataFrame created successfully
DataFrame before reset_index:
                                type of upgrade  \
0  PTO’s\nInterconnection\nFacilities\n(Note 2)   
1                                          None   
2                                          None   
3                                          None   
4                                     

/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_4857/1824975418.py:337: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(self.clean_string_cell)


Starting new table with headers: ['type of upgrade', 'upgrade', 'description', 'cost allocation factor', 'estimated cost x 1000', 'escalated costs x 1000', 'estimated time  to construct']

Processing table with title: Table 7-1: Escalated Cost and Time to Construct for Interconnection Facilities - IF
Current headers: ['type of upgrade', 'upgrade', 'description', 'cost allocation factor', 'estimated cost x 1000', 'escalated costs x 1000', 'estimated time  to construct']
Number of data rows: 4
DataFrame created successfully
DataFrame before reset_index:
                                type of upgrade  \
0  PTO’s\nInterconnection\nFacilities\n(Note 2)   
1                                          None   
2                                          None   
3                                                 

                            upgrade  \
0  230kV Mustang Switching\nStation   
1                   Generation Site   
2                 Transmission Line   
3                             

/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_4857/1824975418.py:337: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(self.clean_string_cell)


In [43]:
import os
import pdfplumber
import pandas as pd
import re
import PyPDF2
from tabulate import tabulate

class PDFTableExtractor:
    def __init__(self):
        pass

    def clean_column_headers(self, headers):
        cleaned_headers = []
        for header in headers:
            if header is None:
                header = ""
            elif isinstance(header, str):
                header = header.lower()
                header = re.sub(r'\s+', ' ', header)
                header = re.sub(r'\(.*?\)', '', header)  # Remove anything in parentheses
                header = re.sub(r'[^a-zA-Z0-9\s]', '', header)
                header = header.strip()
            cleaned_headers.append(header)
        return cleaned_headers

    def clean_string_cell(self, value):
        if isinstance(value, str):
            return value.replace('\n', ' ').strip()  # Remove newlines and extra spaces
        return value

    def contains_phrase(self, row, phrase):
        regex_pattern = re.sub(r"\s+", r"\\s*", phrase)
        pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
        return row.astype(str).apply(lambda cell: bool(pattern.search(cell))).any()

    def search_gps_coordinates(self, text):
        """Search for GPS coordinates using multiple patterns."""
        gps_coords = re.search(r"gps coordinates:\s*([\d\.\-]+),\s*([\d\.\-]+)", text, re.IGNORECASE)
        if gps_coords:
            return gps_coords.groups()

        project_coords = re.search(r"latitude[:\s]*([\d\.\-]+)[^\d]+longitude[:\s]*([\d\.\-]+)", text, re.IGNORECASE)
        if project_coords:
            return project_coords.groups()

        gps_coords_directional = re.search(
            r"gps coordinates:\s*([\d\.\-]+)\s*[nNsS],\s*([\d\.\-]+)\s*[eEwW]", text, re.IGNORECASE)
        if gps_coords_directional:
            lat, lon = gps_coords_directional.groups()
            latitude = lat if "N" in text.upper() else f"-{lat}"  # Adjust latitude sign
            longitude = lon if "E" in text.upper() else f"-{lon}"  # Adjust longitude sign
            return (latitude, longitude)

        return (None, None)

    def extract_pdf_data(self, pdf_path, project_id):
        """Extract base data from the PDF and return as a DataFrame."""
        with open(pdf_path, 'rb') as pdf:
            reader = PyPDF2.PdfReader(pdf)
            text = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text

        text = self.clean_string_cell(text)

        queue_id = re.search(r"q[\s_-]*(\d+)", text, re.IGNORECASE)
        queue_id = queue_id.group(1) if queue_id else str(project_id)  # Use project_id if queue_id is not found

        cluster_number = re.search(r"queue[\s_-]*cluster[\s_-]*(\d+)", text, re.IGNORECASE)
        cluster_number = cluster_number.group(1) if cluster_number else None

        deliverability_status = re.search(r"(\w+)\s*capacity deliverability status", text, re.IGNORECASE)
        deliverability_status = deliverability_status.group(1) if deliverability_status else None

        # Search for GPS coordinates
        latitude, longitude = self.search_gps_coordinates(text)

        capacity = re.search(r"total rated output of (\d+)\s*mw", text, re.IGNORECASE)
        if capacity:
            capacity = int(capacity.group(1))
        else:
            capacity2 = re.search(r"(\d+)\s*mw", text)
            capacity = int(capacity2.group(1)) if capacity2 else None

        # Initialize base data dictionary
        base_data = {
            "q_id": [queue_id],
            "cluster": [cluster_number],
            "req_deliverability": [deliverability_status],
            "latitude": [latitude],
            "longitude": [longitude],
            "capacity": [capacity],
        }

        return pd.DataFrame(base_data)

    def extract_table7_and_replace_none(self, pdf_path, project_id):
        """Extract all tables from pages containing 'Table 7' and process them."""
        base_data = self.extract_pdf_data(pdf_path, project_id)
        pdf_core = pd.DataFrame()

        with pdfplumber.open(pdf_path) as pdf:
            pages_with_table7 = []
            table_titles = []
            for page_number, page in enumerate(pdf.pages):
                text = page.extract_text()
                if text and 'Table 7' in text:
                    pages_with_table7.append(page_number)
                    # Extract the last table title on this page
                    lines = text.split('\n')
                    for line in lines:
                        if 'Table 7' in line:
                            table_title = self.find_table_title(line, "Table 7")
                            table_titles.append(table_title)

            # Use the last table title found for 'type of upgrade'
            if table_titles:
                last_table_title = table_titles[-1]
                last_title_pattern = self.search_in_table_title(
                    last_table_title,
                    ["PTO", "Reliability Network Upgrade", "ADNU", "LDNU", "RNU", "Area Delivery Network Upgrade", "Local Delivery Network"]
                )
            else:
                last_title_pattern = None

            # Scrape all tables on the pages containing 'Table 7'
            for page_number in pages_with_table7:
                page = pdf.pages[page_number]
                tables = page.find_tables(table_settings={
                    "horizontal_strategy": "lines",
                    "vertical_strategy": "lines",
                })
                for table in tables:
                    tab = table.extract()
                    if len(tab) > 1:
                        headers = self.clean_column_headers(tab[0])
                        data_rows = tab[1:]
                        # Ensure headers are unique
                        headers = self.make_unique_headers(headers)
                        df_table = pd.DataFrame(data_rows, columns=headers)
                        df_table.reset_index(drop=True, inplace=True)
                        # Insert 'type of upgrade' column
                        if 'type of upgrade' in df_table.columns:
                            df_table['type of upgrade'] = df_table['type of upgrade'].fillna(last_title_pattern)
                        else:
                            df_table.insert(0, 'type of upgrade', last_title_pattern)
                        pdf_core = pd.concat([pdf_core, df_table], ignore_index=True)

        # Join the base data with the scraped table data
        if not pdf_core.empty:
            overlapping_columns = base_data.columns.intersection(pdf_core.columns)
            pdf_core = pdf_core.drop(columns=overlapping_columns)
            merged_df = base_data.join(pdf_core)
        else:
            # If no valid tables were found, return base data only
            merged_df = base_data

        return merged_df

    def make_unique_headers(self, headers):
        """Ensure all headers are unique and not empty."""
        unique_headers = []
        seen = {}
        for i, header in enumerate(headers):
            if not header or header in seen:
                header = f"column_{i}"
            seen[header] = seen.get(header, 0) + 1
            if seen[header] > 1:
                header = f"{header}_{seen[header]-1}"
            unique_headers.append(header)
        return unique_headers

    @staticmethod
    def find_table_title(line, table_identifier):
        table_title_match = re.search(rf"({table_identifier}[^\n]*)", line)
        return table_title_match.group(1).strip() if table_title_match else None

    @staticmethod
    def search_in_table_title(table_title, patterns):
        for pattern in patterns:
            if re.search(pattern, table_title, re.IGNORECASE):
                return pattern
        return table_title  # Return the full title if no pattern matches

    def process_pdfs(self, pdf_list):
        all_dfs = []  # List to collect DataFrames from all PDFs
        for pdf_path in pdf_list:
            if os.path.exists(pdf_path) and pdf_path.endswith(".pdf"):
                try:
                    print(f"\nProcessing {pdf_path}")
                    # Extract project_id from pdf_path if needed
                    project_id = os.path.basename(pdf_path).split('.')[0]  # Modify as appropriate
                    df = self.extract_table7_and_replace_none(pdf_path, project_id)
                    if not df.empty:
                        # Clean up the DataFrame by cleaning string cells
                        df = df.map(self.clean_string_cell)
                        # Optionally drop rows containing specific phrases
                        df = df[~df.apply(lambda row: self.contains_phrase(row, "Type of Upgrade"), axis=1)]
                        all_dfs.append(df)
                    else:
                        print(f"No data extracted from {pdf_path}")
                except Exception as e:
                    print(f"Error processing {pdf_path}: {e}")
            else:
                print(f"File not found or invalid: {pdf_path}")

        # Combine all DataFrames into one
        if all_dfs:
            combined_df = pd.concat(all_dfs, ignore_index=True)
            # Print the combined DataFrame using tabulate
            print("\nCombined extracted data:")
            print(tabulate(combined_df, headers='keys', tablefmt='fancy_grid'))
            # Optionally, save the combined DataFrame to a CSV or Excel file
            combined_df.to_csv('combined_output.csv', index=False)
            # combined_df.to_excel('combined_output.xlsx', index=False)
        else:
            print("No data extracted from any PDF.")

# Usage example
pdf_list = [
    "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1030/02_phase_1_study/C7PhI - Appendix A - Q1030   South Lake Solar.pdf",
    "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data/1158/02_phase_1_study/Q1158 Slate_Appendix A Report_C8PhI.pdf",
    # Add more PDF paths as needed
]

extractor = PDFTableExtractor()
extractor.process_pdfs(pdf_list)



Processing /Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/1030/02_phase_1_study/C7PhI - Appendix A - Q1030   South Lake Solar.pdf

Processing /Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/data/1158/02_phase_1_study/Q1158 Slate_Appendix A Report_C8PhI.pdf

Combined extracted data:
╒════╤════════╤═══════════╤══════════════════════╤════════════╤═════════════╤════════════╤═══════════════════════════════════════════╤═════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤══════════════════════════╤═════════════════════════╤══════════════════════════╤════════════════════════════════╤════════════╤═══════════════════════════════════╤══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤═════════╤══════╤════════════╤══════╕
│    │   q_id │   cluster │ req_d

# Data Cleaner code

In [14]:
import pandas as pd
import re
import unicodedata

# Load the CSV file
df = pd.read_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/c7_data.csv', dtype={'estimated_time_to_construct': str})


 
 



def convert_to_snake_case(column_name):
    column_name = column_name.strip().lower()
    column_name = re.sub(r'[\s\-]+', '_', column_name)
    column_name = re.sub(r'[^\w]', '', column_name)
    return column_name


def clean_string_cell(value):
    if isinstance(value, str):
        # Normalize Unicode to NFKD and remove non-ASCII characters
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
        # Remove newlines and extra spaces
        value = value.replace('\n', ' ').strip()
    return value


df = df.map(clean_string_cell)



# Apply the function to all columns
df.columns = [convert_to_snake_case(col) for col in df.columns]

 
 






# Step 1: Create the 'item' column based on whether the 'type_of_upgrade' row or 'cost_allocation_factor' contains 'Total'
df['item'] = df.apply(
    lambda row: 'no' if (pd.notna(row['type_of_upgrade']) and 'Total' in str(row['type_of_upgrade'])) or 
                         (pd.notna(row['cost_allocation_factor']) and 'Total' in str(row['cost_allocation_factor']))
               else 'yes', axis=1)

# Step 2: Move 'item' column next to 'type_of_upgrade' column
cols = df.columns.tolist()
item_index = cols.index('item')
cols.insert(cols.index('type_of_upgrade') + 1, cols.pop(item_index))
df = df[cols]

# Step 3: Remove the 'Total' values from the 'cost_allocation_factor' column if they are already in the 'type_of_upgrade' column
df['cost_allocation_factor'] = df.apply(
    lambda row: None if pd.notna(row['type_of_upgrade']) and 'Total' in str(row['type_of_upgrade']) else row['cost_allocation_factor'], axis=1
)

# Step 4: For each `q_id` and `type_of_upgrade`, if it has only one row and no 'Total' is present in 'cost_allocation_factor', create a new `Total` row
new_rows = []
for q_id, group in df.groupby('q_id'):
    unique_upgrades = group['type_of_upgrade'].unique()

    # Check if there's already a "Total" in 'cost_allocation_factor' for this q_id
    if any('Total' in str(x) for x in group['cost_allocation_factor']):
        continue  # If Total exists under 'cost_allocation_factor', skip creating new total row for this q_id
    
    for upgrade in unique_upgrades:
        # Skip if "Total" is already present for this upgrade or if the upgrade is NaN
        if pd.isna(upgrade) or 'Total' in str(upgrade):
            continue
        
        # Get rows corresponding to this specific upgrade
        rows = group[group['type_of_upgrade'] == upgrade]
        
        if len(rows) == 1:
            # Duplicate the row and create a new row with "Total" under 'type_of_upgrade'
            original_row = rows.iloc[0].copy()
            total_row = original_row.copy()

            # Modify the total_row to reflect "Total"
            total_row['type_of_upgrade'] = 'Total'
            total_row['item'] = 'no'

            # Append the Total row immediately after the original row
            original_index = df[(df['q_id'] == q_id) & (df['type_of_upgrade'] == upgrade)].index[0]
            new_rows.append((original_index + 1, total_row))

# Step 5: Insert the new rows into the DataFrame in the correct order
for idx, row in sorted(new_rows, reverse=True):
    df = pd.concat([df.iloc[:idx], pd.DataFrame([row]), df.iloc[idx:]]).reset_index(drop=True)

# Step 6: Move "Total" from 'cost_allocation_factor' column to 'type_of_upgrade' column
df['type_of_upgrade'] = df.apply(
    lambda row: 'Total' if 'Total' in str(row['cost_allocation_factor']) and 'Total' not in str(row['type_of_upgrade']) else row['type_of_upgrade'],
    axis=1
)

# Step 7: After moving "Total" to 'type_of_upgrade', clear it from 'cost_allocation_factor'
df['cost_allocation_factor'] = df.apply(
    lambda row: None if 'Total' in str(row['cost_allocation_factor']) else row['cost_allocation_factor'],
    axis=1
)


def clean_estimated_time(value):
    if isinstance(value, str):
        value = re.sub(r'\s*months?\s*', '', value, flags=re.IGNORECASE).strip()
    return value
df['estimated_time_to_construct'] = df['estimated_time_to_construct'].apply(clean_estimated_time)



df['type_of_upgrade'] = df['type_of_upgrade'].apply(lambda x: re.sub(r'\s*\(note \d+\)', '', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x)




mappings = {
     'Reliability Network Upgrade': 'RNU',
     'Reliability Network upgrade': 'RNU',
     'Local Delivery Network Upgrade': 'LDNU',
     "PTOs Interconnection Facilities":'PTO_IF',
     'Area Delivery Network Upgrade':'ADNU',
     
}

# Apply transformations using a lambda function
df['type_of_upgrade'] = df['type_of_upgrade'].apply(lambda x: re.sub(r'\bupgrades\b', 'upgrade', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x)
df['type_of_upgrade'] = df['type_of_upgrade'].apply(lambda x: mappings.get(x, x) if isinstance(x, str) else x)
#df['type_of_upgrade'] = df['type_of_upgrade'].str.title()
# Step 8: Forward fill the non-empty values in 'type_of_upgrade' to replace empty cells with the previous value
df['type_of_upgrade'] = df['type_of_upgrade'].ffill()


df['upgrade'] = df.groupby(['q_id','type_of_upgrade'])['upgrade'].ffill()

 


 

 

# Step 9: Ensure 'Total' is correctly replaced in 'type_of_upgrade' when present
previous_type_of_upgrade = None

for i in range(len(df)):
    if df.at[i, 'type_of_upgrade'] == 'Total':
        if previous_type_of_upgrade is not None:
            df.at[i, 'type_of_upgrade'] = previous_type_of_upgrade
    else:
        previous_type_of_upgrade = df.at[i, 'type_of_upgrade']

# Step 10: Fill NaN in descriptive columns with None and NaN in numeric columns with 0
# List of numeric columns and non-numeric columns based on dataset structure
numeric_columns = ['cost_allocation_factor',  'estimated_cost_x_1000', 'estimated_time_to_construct', 'total_estimated_cost_x_1000_escalalted', 'adnu_cost_rate_x_1000', 'escalated_cost_x_1000', ]
non_numeric_columns = ['type_of_upgrade', 'upgrade', 'description']

for col in non_numeric_columns:
     df[col] = df[col].apply(lambda x: 'None' if pd.isna(x) else x)

# Replace '-' with NaN in numeric columns (so they can be converted to 0)
df[numeric_columns] = df[numeric_columns].replace('-', pd.NA)     

# Replace NaN in numeric columns with 0
df[numeric_columns] = df[numeric_columns].fillna(0)

# Step 11: Sort the DataFrame by q_id and type_of_upgrade
df['original_order'] = df.index
df = df.sort_values(by=['q_id', 'original_order'], ascending=[True, True])

# Optional: Drop the original_order column if not needed anymore
df = df.drop(columns=['original_order'])


#df = df.sort_values(by=['q_id', 'type_of_upgrade', 'upgrade', 'description'], ascending=[True, False, False, False])

# Step 12: Create the itemized dataset (rows where item == 'yes') and save to CSV
itemized_df = df[df['item'] == 'yes']
itemized_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/c7_itemized.csv', index=False)

# Step 13: Create the totals dataset (rows where item == 'no') and drop unwanted columns
totals_df = df[df['item'] == 'no'].drop(columns=['upgrade', 'description', 'cost_allocation_factor', 'estimated_time_to_construct'])
totals_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/c7_totals.csv', index=False)

print(f"Itemized rows saved to 'itemized.csv'.")
print(f"Filtered Total rows saved to 'totals.csv'.")

print(df['type_of_upgrade'].unique())


Itemized rows saved to 'itemized.csv'.
Filtered Total rows saved to 'totals.csv'.
['PTO_IF' 'RNU' 'LDNU' 'ADNU']


# Data cleaner for clusters 6,7,8

In [4]:
import pandas as pd
import re
import unicodedata

# Load the CSV file
df = pd.read_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/C_6_7_8_data.csv', dtype={'estimated_time_to_construct': str})


def merge_columns(df):
    merge_columns_dict = {
            "escalated cost x 1000": ["escalated costs x 1000", "estimated cost x 1000 escalated" , "estimated cost x 1000 escalated with itcca"],
            "estimated time to construct": ["estimated time to construct", "estimated time  to construct"],
            "total estimated cost x 1000 escalalted": ["total estimated cost x 1000 escalalted", "total estimated cost x 1000 escalated"],
            "adnu cost rate x 1000": ["adnu cost rate x 1000", "cost rate x 1000"],
            "description": ["description"],
            "capacity": ["capacity", "project size"],
            "cost allocation factor": ["cost allocation factor", "cost allocatio n factor"],


        }

    # Identify unnamed columns (columns with no name or those starting with "Unnamed")
    unnamed_columns = [col for col in df.columns if pd.isna(col) or col.strip() == "" or col.startswith("Unnamed")]

    if unnamed_columns:
        # Merge unnamed columns into 'description' if they exist
        merge_columns_dict["description"].extend(unnamed_columns)

    # Iterate over the dictionary to merge columns
    for new_col, old_cols in merge_columns_dict.items():
        if any(col in df.columns for col in old_cols):
            # Create a new column by backfilling data from the old columns
            df[new_col] = df[old_cols].bfill(axis=1).iloc[:, 0]
            # Drop the old columns, except for the new one we just created
            df.drop(columns=[col for col in old_cols if col in df.columns and col != new_col], inplace=True)

    return df


     

# Apply the merge columns function to the DataFrame
df = merge_columns(df) 
 



def convert_to_snake_case(column_name):
    column_name = column_name.strip().lower()
    column_name = re.sub(r'[\s\-]+', '_', column_name)
    column_name = re.sub(r'[^\w]', '', column_name)
    return column_name


def clean_string_cell(value):
    if isinstance(value, str):
        # Normalize Unicode to NFKD and remove non-ASCII characters
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
        # Remove newlines and extra spaces
        value = value.replace('\n', ' ').strip()
    return value


df = df.map(clean_string_cell)



# Apply the function to all columns
df.columns = [convert_to_snake_case(col) for col in df.columns]

 
 






# Step 1: Create the 'item' column based on whether the 'type_of_upgrade' row or 'cost_allocation_factor' contains 'Total'
df['item'] = df.apply(
    lambda row: 'no' if (pd.notna(row['type_of_upgrade']) and 'Total' in str(row['type_of_upgrade'])) or 
                         (pd.notna(row['cost_allocation_factor']) and 'Total' in str(row['cost_allocation_factor']))
               else 'yes', axis=1)

# Step 2: Move 'item' column next to 'type_of_upgrade' column
cols = df.columns.tolist()
item_index = cols.index('item')
cols.insert(cols.index('type_of_upgrade') + 1, cols.pop(item_index))
df = df[cols]

# Step 3: Remove the 'Total' values from the 'cost_allocation_factor' column if they are already in the 'type_of_upgrade' column
df['cost_allocation_factor'] = df.apply(
    lambda row: None if pd.notna(row['type_of_upgrade']) and 'Total' in str(row['type_of_upgrade']) else row['cost_allocation_factor'], axis=1
)

# Step 4: For each `q_id` and `type_of_upgrade`, if it has only one row and no 'Total' is present in 'cost_allocation_factor', create a new `Total` row
new_rows = []
for q_id, group in df.groupby('q_id'):
    unique_upgrades = group['type_of_upgrade'].unique()

    # Check if there's already a "Total" in 'cost_allocation_factor' for this q_id
    if any('Total' in str(x) for x in group['cost_allocation_factor']):
        continue  # If Total exists under 'cost_allocation_factor', skip creating new total row for this q_id
    
    for upgrade in unique_upgrades:
        # Skip if "Total" is already present for this upgrade or if the upgrade is NaN
        if pd.isna(upgrade) or 'Total' in str(upgrade):
            continue
        
        # Get rows corresponding to this specific upgrade
        rows = group[group['type_of_upgrade'] == upgrade]
        
        if len(rows) == 1:
            # Duplicate the row and create a new row with "Total" under 'type_of_upgrade'
            original_row = rows.iloc[0].copy()
            total_row = original_row.copy()

            # Modify the total_row to reflect "Total"
            total_row['type_of_upgrade'] = 'Total'
            total_row['item'] = 'no'

            # Append the Total row immediately after the original row
            original_index = df[(df['q_id'] == q_id) & (df['type_of_upgrade'] == upgrade)].index[0]
            new_rows.append((original_index + 1, total_row))

# Step 5: Insert the new rows into the DataFrame in the correct order
for idx, row in sorted(new_rows, reverse=True):
    df = pd.concat([df.iloc[:idx], pd.DataFrame([row]), df.iloc[idx:]]).reset_index(drop=True)

# Step 6: Move "Total" from 'cost_allocation_factor' column to 'type_of_upgrade' column
df['type_of_upgrade'] = df.apply(
    lambda row: 'Total' if 'Total' in str(row['cost_allocation_factor']) and 'Total' not in str(row['type_of_upgrade']) else row['type_of_upgrade'],
    axis=1
)

# Step 7: After moving "Total" to 'type_of_upgrade', clear it from 'cost_allocation_factor'
df['cost_allocation_factor'] = df.apply(
    lambda row: None if 'Total' in str(row['cost_allocation_factor']) else row['cost_allocation_factor'],
    axis=1
)


def clean_estimated_time(value):
    if isinstance(value, str):
        value = re.sub(r'\s*months?\s*', '', value, flags=re.IGNORECASE).strip()
    return value
df['estimated_time_to_construct'] = df['estimated_time_to_construct'].apply(clean_estimated_time)



df['type_of_upgrade'] = df['type_of_upgrade'].apply(lambda x: re.sub(r'\s*\(note \d+\)', '', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x)




mappings = {
     'Reliability Network Upgrade': 'RNU',
     'Reliability Network upgrade': 'RNU',
     'Local Delivery Network Upgrade': 'LDNU',
     'Local Delivery Network': 'LDNU',
     "PTOs Interconnection Facilities":'PTO_IF',
     'Area Delivery Network Upgrade':'ADNU',
     'Reliability Network upgrade to Physically Interconnect': 'RNU',
     'PTO' : 'PTO_IF',

     
}

# Apply transformations using a lambda function
df['type_of_upgrade'] = df['type_of_upgrade'].apply(lambda x: re.sub(r'\bupgrades\b', 'upgrade', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x)
df['type_of_upgrade'] = df['type_of_upgrade'].apply(lambda x: mappings.get(x, x) if isinstance(x, str) else x)
#df['type_of_upgrade'] = df['type_of_upgrade'].str.title()
# Step 8: Forward fill the non-empty values in 'type_of_upgrade' to replace empty cells with the previous value
df['type_of_upgrade'] = df['type_of_upgrade'].ffill()


df['upgrade'] = df.groupby(['q_id','type_of_upgrade'])['upgrade'].ffill()

 


 

 

# Step 9: Ensure 'Total' is correctly replaced in 'type_of_upgrade' when present
previous_type_of_upgrade = None

for i in range(len(df)):
    if df.at[i, 'type_of_upgrade'] == 'Total':
        if previous_type_of_upgrade is not None:
            df.at[i, 'type_of_upgrade'] = previous_type_of_upgrade
    else:
        previous_type_of_upgrade = df.at[i, 'type_of_upgrade']

# Step 10: Fill NaN in descriptive columns with None and NaN in numeric columns with 0
# List of numeric columns and non-numeric columns based on dataset structure
numeric_columns = ['cost_allocation_factor',  'estimated_cost_x_1000', 'estimated_time_to_construct', 'total_estimated_cost_x_1000_escalalted', 'adnu_cost_rate_x_1000', 'escalated_cost_x_1000',
                    'estimated_cost_x_1000_escalated_without_itcca', 'adnu_cost_rate_x_1000_escalated']
non_numeric_columns = ['type_of_upgrade', 'upgrade', 'description']

for col in non_numeric_columns:
     df[col] = df[col].apply(lambda x: 'None' if pd.isna(x) else x)

# Replace '-' with NaN in numeric columns (so they can be converted to 0)
df[numeric_columns] = df[numeric_columns].replace('-', pd.NA)     

# Replace NaN in numeric columns with 0
df[numeric_columns] = df[numeric_columns].fillna(0)

# Step 11: Sort the DataFrame by q_id and type_of_upgrade
df['original_order'] = df.index
df = df.sort_values(by=['q_id', 'original_order'], ascending=[True, True])

# Optional: Drop the original_order column if not needed anymore
df = df.drop(columns=['original_order'])

#df = df.drop(columns=['project_parameters', 'project_specific_data'])

#df = df.sort_values(by=['q_id', 'type_of_upgrade', 'upgrade', 'description'], ascending=[True, False, False, False])

# Step 12: Create the itemized dataset (rows where item == 'yes') and save to CSV
itemized_df = df[df['item'] == 'yes']
itemized_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/costs_phase_1_cluster_678_itemized.csv', index=False)

# Step 13: Create the totals dataset (rows where item == 'no') and drop unwanted columns
totals_df = df[df['item'] == 'no'].drop(columns=['upgrade', 'description', 'cost_allocation_factor', 'estimated_time_to_construct'])
totals_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 7/costs_phase_1_cluster_678_total.csv', index=False)

print(f"Itemized rows saved to 'itemized.csv'.")
print(f"Filtered Total rows saved to 'totals.csv'.")

print(df['type_of_upgrade'].unique())


/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_12248/290742439.py:33: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[new_col] = df[old_cols].bfill(axis=1).iloc[:, 0]
/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_12248/290742439.py:33: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[new_col] = df[old_cols].bfill(axis=1).iloc[:, 0]
/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_12248/290742439.py:33: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future ve

Itemized rows saved to 'itemized.csv'.
Filtered Total rows saved to 'totals.csv'.
['ADNU' 'PTO_IF' 'RNU' 'LDNU'
 'Add Two (2) Pittsburg PP to Contra Costa PP 230 kV circuits'
 'Convert Pittsburg PP 230 kV bus section D to 6 bay GIS BAAH'
 'Add two (2) circuit elements at Contra Costa PP 230 kV'
 'Re-conductor Contra Costa PP-Contra Costa Sub 230 kV line']


In [359]:
import os
import shutil

# Set the directory
new_directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/"
os.chdir(new_directory)

pdf_dir = "data"  # Replace with your actual directory

# Iterate through each project folder inside pdf_dir
for project_folder in os.listdir(pdf_dir):
    project_path = os.path.join(pdf_dir, project_folder)
    
    if os.path.isdir(project_path):
        phase_1_study_folder = os.path.join(project_path, '02_phase_1_study')
        phase_2_study_folder = os.path.join(project_path, '03_phase_2_study')
        reassessment_folder = os.path.join(project_path, '05_reassesment')

        # Ensure the reassessment folder exists
        os.makedirs(reassessment_folder, exist_ok=True)

        # Check each PDF in the Phase 1 folder
        for study_folder in [phase_1_study_folder, phase_2_study_folder]:
            if os.path.exists(study_folder):
                for filename in os.listdir(study_folder):
                    file_path = os.path.join(study_folder, filename)
                    if filename.endswith(".pdf") and 'reassessment' in filename.lower():
                        # Move the PDF to the reassessment folder
                        shutil.move(file_path, os.path.join(reassessment_folder, filename))
                        print(f"Moved {filename} to {reassessment_folder}")


Moved Q1432_Starlight Solar_Reassessment Appendix A_7-31-2020_SDGE.pdf to data/1432/05_reassesment
Moved QC12 Ph2 Attachment 2 Q1631 TOT909 Reassessment 2022.pdf to data/1631/05_reassesment
Moved 2021-Reassessment-SCE-Eastern-Q1636-Calypso-Appendix A-Attachment 1.pdf to data/1636/05_reassesment
Moved 2021-Reassessment-SCE-Eastern-Q1636-Calypso-Appendix A-Attachment 2.pdf to data/1636/05_reassesment
Moved 2021-Reassessment-SCE-Eastern-Q1636-Calypso-Appendix A.pdf to data/1636/05_reassesment
Moved 2019-Reassessment Report-PGE Fresno Interconnection Area-Appendix I.pdf to data/1239/05_reassesment
Moved Q1239_Medeiros Solar_Appendix A_2019 Reassessment Report_Addendum.pdf to data/1239/05_reassesment
Moved Q1691-Fortis_Appendix A_2022 Reassessment_Report_Addendum1.pdf to data/1691/05_reassesment
Moved Q1495-Chalan Solar_Appendix A_2022 Reassessment_Report_Addendum3.pdf to data/1495/05_reassesment
Moved Q1495_C11PhII_Chalan Solar_2020 Reassessment Report_Appendix A_Addendum.pdf to data/1495/

In [365]:
import os
import shutil
import re

# Set the directory
new_directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/"
os.chdir(new_directory)

pdf_dir = "data"  # Replace with your actual directory

# Create regex pattern for Appendix and Attachment matching
appendix_pattern = re.compile(r'appendix\s*[B-K]', re.IGNORECASE)
attachment_pattern = re.compile(r'attachment\s*(?:[3-9]|10|11)', re.IGNORECASE)
deliverability_pattern = re.compile(r'deliverability assessment', re.IGNORECASE)

# Iterate through each project folder inside pdf_dir
for project_folder in os.listdir(pdf_dir):
    project_path = os.path.join(pdf_dir, project_folder)
    
    if os.path.isdir(project_path):
        phase_1_study_folder = os.path.join(project_path, '02_phase_1_study')
        phase_2_study_folder = os.path.join(project_path, '03_phase_2_study')
        document_dump_folder = os.path.join(project_path, 'document_dump')

        # Ensure the document dump folder exists
        os.makedirs(document_dump_folder, exist_ok=True)

        # Check each PDF in the Phase 1 and Phase 2 folders
        for study_folder in [phase_1_study_folder, phase_2_study_folder]:
            if os.path.exists(study_folder):
                for filename in os.listdir(study_folder):
                    file_path = os.path.join(study_folder, filename)
                    if filename.endswith(".pdf"):
                        # Check if the filename contains Appendix B-L, Attachment 3-11, or deliverability assessments
                        if appendix_pattern.search(filename) or attachment_pattern.search(filename) or (study_folder == phase_1_study_folder and deliverability_pattern.search(filename)):
                            # Move the PDF to the document dump folder
                            shutil.move(file_path, os.path.join(document_dump_folder, filename))
                            print(f"Moved {filename} to {document_dump_folder}")

Moved Appendix I - Transient Stability Plots.pdf to data/1432/document_dump
Moved Appendix I - Deliverability Assessment Results_South 500kV_C12PhI.pdf to data/1600/document_dump
Moved QC14PII-SCE-Northern-Area Report-Appendix I-Northern Hemisphere Nomogram.pdf to data/2085/document_dump
Moved Appendix I - Deliverability Assessment Results_Kern_C9PhI.pdf to data/1264/document_dump
Moved Appendix I - Deliverability Assessment Results_Kern_C9PhI.pdf to data/1263/document_dump
Moved Appendix I - Transient Stability Plots.pdf to data/1435/document_dump
Moved QC12PI-SCE-Area Report-Appendix I-Northern Hemisphere Nomogram.pdf to data/1631/document_dump
Moved QC12PII-SCE-Area Report-Appendix I-Northern Hemisphere Nomogram.pdf to data/1631/document_dump
Moved C10PhI-SCE-Eastern-Area Report-Appendix I-Northern Hemisphere Nomogram.pdf to data/1403/document_dump
Moved QC10PII-SCE-Area Report-Appendix I-Northern Hemisphere Nomogram.pdf to data/1403/document_dump
Moved Appendix I - Deliverability A

In [366]:
import os
import shutil

# Set the directory
new_directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/"
os.chdir(new_directory)

pdf_dir = "data"  # Replace with your actual directory

# Create a regex pattern for "reassessment"
reassessment_pattern = re.compile(r'reassessment', re.IGNORECASE)

# Iterate through each project folder inside pdf_dir
for project_folder in os.listdir(pdf_dir):
    project_path = os.path.join(pdf_dir, project_folder)
    
    if os.path.isdir(project_path):
        document_dump_folder = os.path.join(project_path, 'document_dump')
        reassessment_folder = os.path.join(project_path, '05_reassesment')

        # Ensure the reassessment folder exists
        os.makedirs(reassessment_folder, exist_ok=True)

        # Check each PDF in the document_dump folder
        if os.path.exists(document_dump_folder):
            for filename in os.listdir(document_dump_folder):
                file_path = os.path.join(document_dump_folder, filename)
                if filename.endswith(".pdf") and reassessment_pattern.search(filename):
                    # Move the PDF to the reassessment folder
                    shutil.move(file_path, os.path.join(reassessment_folder, filename))
                    print(f"Moved {filename} to {reassessment_folder}")


Moved 14AS870347-Q1069__Boulder_Solar_3_fka_Boulder_Solar_Power__2016_Reassessment_Cover_Letter.pdf to data/1069/05_reassesment
Moved 2020 Reassessment_SDGE Area Report_7-31-2020_SDGE.pdf to data/1432/05_reassesment
Moved Q1432_Starlight_Solar_2022_Reassessment_Report-7-29-22.pdf to data/1432/05_reassesment
Moved Q1432_Starlight Solar_C10PhII_Reassessment Cover Letter.pdf to data/1432/05_reassesment
Moved 2022_Reassessment_SDGE_Area_Report_7-29-22.pdf to data/1432/05_reassesment
Moved 2022-Reassessment-SCE-Northern-Area Report-clean.pdf to data/1631/05_reassesment
Moved Appendix B - PGE Area System Assumptions-2015 Reassessment.pdf to data/799/05_reassesment
Moved C4_Q799_Aspiration Solar D_2015 Reassessment Cover Letter.pdf to data/799/05_reassesment
Moved 10AS666915-Reassessment_Report_SCE_East_of_Pisgah__Final.pdf to data/593/05_reassesment
Moved 10AS666915-2014ReassessmentSCEEOPArea_Report.pdf to data/593/05_reassesment
Moved 10AS666915-Q593_Fort_Mojave_Solar_Project__Cover_Letter2

In [367]:
import os
import shutil
import re

# Set the directory
new_directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/"
os.chdir(new_directory)

pdf_dir = "data"  # Replace with your actual directory

# Create a regex pattern to match various forms of PH2 or PHII
ph2_pattern = re.compile(r'ph[\s_-]*(2|II)', re.IGNORECASE)

# Iterate through each project folder inside pdf_dir
for project_folder in os.listdir(pdf_dir):
    project_path = os.path.join(pdf_dir, project_folder)
    
    if os.path.isdir(project_path):
        phase_1_study_folder = os.path.join(project_path, '02_phase_1_study')
        phase_2_study_folder = os.path.join(project_path, '03_phase_2_study')

        # Ensure the phase 2 folder exists
        os.makedirs(phase_2_study_folder, exist_ok=True)

        # Check each PDF in the Phase 1 folder
        if os.path.exists(phase_1_study_folder):
            for filename in os.listdir(phase_1_study_folder):
                file_path = os.path.join(phase_1_study_folder, filename)
                if filename.endswith(".pdf") and ph2_pattern.search(filename):
                    # Move the PDF to the Phase 2 study folder
                    shutil.move(file_path, os.path.join(phase_2_study_folder, filename))
                    print(f"Moved {filename} to {phase_2_study_folder}")


Moved QC10PhII_Q1432_Starlight Solar_Appendix A_11-21-2018_final.pdf to data/1432/03_phase_2_study
Moved QC10PhII_Q1435_Viktoria Solar_Addm1toAppendix A_1-22-2019.pdf to data/1435/03_phase_2_study
Moved QC10PhII_Q1435_Viktoria Solar_Appendix A_11-21-2018_final.pdf to data/1435/03_phase_2_study
Moved QC12PhII_Q1662_Ventasso_Energy_Storage_Appendix_A_final_11-20-2020.pdf to data/1662/03_phase_2_study
Moved Q1239 Medeiros Solar_C9PhII_Appendix A-Addendum1.pdf to data/1239/03_phase_2_study
Moved Q1691FortisAppendix_AC13PhIIRevision1.pdf to data/1691/03_phase_2_study
Moved QC12PhII_Q1665_Kingsley_Solar_Farm_Appendix_A_final_11-20-2020.pdf to data/1665/03_phase_2_study
Moved QC12PhII_Q1665_Kingsley_Solar_Farm_Appendix_A_final_11-20-2020_v2.pdf to data/1665/03_phase_2_study
Moved Q1495_Chalan_Solar_Appendix_AC11PhIIRevision1.pdf to data/1495/03_phase_2_study
Moved Q1461_Reclaimed_Wind_Appendix_AC11PhIIRevision1.pdf to data/1461/03_phase_2_study
Moved QC13PhII_Q1806_Captiva_Energy_Storage_Appe

In [369]:
import os
import shutil
import re

# Set the directory
new_directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/"
os.chdir(new_directory)

pdf_dir = "data"  # Replace with your actual directory

# Create regex patterns to match Area, Cover, and Appendix B-J
area_cover_pattern = re.compile(r'(area|cover)', re.IGNORECASE)
appendix_pattern = re.compile(r'appendix\s*[B-J]', re.IGNORECASE)

# Iterate through each project folder inside pdf_dir
for project_folder in os.listdir(pdf_dir):
    project_path = os.path.join(pdf_dir, project_folder)
    
    if os.path.isdir(project_path):
        reassessment_folder = os.path.join(project_path, '05_reassesment')
        document_dump_folder = os.path.join(project_path, 'document_dump')

        # Ensure the document dump folder exists
        os.makedirs(document_dump_folder, exist_ok=True)

        # Check each PDF in the reassessment folder
        if os.path.exists(reassessment_folder):
            for filename in os.listdir(reassessment_folder):
                file_path = os.path.join(reassessment_folder, filename)
                if filename.endswith(".pdf"):
                    # Check if the filename contains Area, Cover, or Appendix B-J
                    if area_cover_pattern.search(filename) or appendix_pattern.search(filename):
                        # Move the PDF to the document dump folder
                        shutil.move(file_path, os.path.join(document_dump_folder, filename))
                        print(f"Moved {filename} to {document_dump_folder}")


Moved Q569-C1C2 Phase II Final Study Report-Revision-2_Cover Letter_CMB 13JAN2012.pdf to data/569/document_dump
Moved QC7PhII_Q1058_Pio Pico Capacity Increase_Addm1_Cover Letter.pdf to data/1058/document_dump
Moved QC7PhII_Q1058_Pio Pico Capacity Increase_Addm2_Cover Letter.pdf to data/1058/document_dump
Moved Q1206 – Tampadero Solar  Project_Cover Letter_C8 PHII.pdf to data/1206/document_dump
Moved C13PhII PGE Greater Bay Area Interconnection Area Report.pdf to data/1691/document_dump
Moved Q1208 Antelope 2 Project_Cover Letter_C8 PHII.pdf to data/1208/document_dump
Moved Q1664_Cucapa Energia_C12PhII_Cover Letter.pdf to data/1664/document_dump
Moved C8P2-SCE-Eastern-Q1200 GÇô Victory Pass Solar Project_Cover Letter_C8 PHII.pdf to data/1200/document_dump
Moved C8P2-SCE-Eastern-Q1200 – Victory Pass Solar Project_Cover Letter_C8 PHII.pdf to data/1200/document_dump
Moved C13PhII PGE Greater Bay Area Interconnection Area Report.pdf to data/1690/document_dump
Moved Q1207 YUCCA Solar Farm 

In [370]:
import os
import shutil
import re

# Set the directory
new_directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/"
os.chdir(new_directory)

pdf_dir = "data"  # Replace with your actual directory

# Create regex patterns for AppdxA (Appendix A) and Attch1 (Attachment 1)
appdxA_pattern = re.compile(r'appdx\s*a', re.IGNORECASE)
attch1_pattern = re.compile(r'attch\s*1', re.IGNORECASE)

# Iterate through each project folder inside pdf_dir
for project_folder in os.listdir(pdf_dir):
    project_path = os.path.join(pdf_dir, project_folder)
    
    if os.path.isdir(project_path):
        document_dump_folder = os.path.join(project_path, 'document_dump')
        phase_2_study_folder = os.path.join(project_path, '03_phase_2_study')

        # Ensure the Phase 2 study folder exists
        os.makedirs(phase_2_study_folder, exist_ok=True)

        # Check each PDF in the document_dump folder
        if os.path.exists(document_dump_folder):
            for filename in os.listdir(document_dump_folder):
                file_path = os.path.join(document_dump_folder, filename)
                if filename.endswith(".pdf"):
                    # Check if the filename contains AppdxA or Attch1
                    if appdxA_pattern.search(filename) or attch1_pattern.search(filename):
                        # Move the PDF to the Phase 2 study folder
                        shutil.move(file_path, os.path.join(phase_2_study_folder, filename))
                        print(f"Moved {filename} to {phase_2_study_folder}")


Moved C8P2-NOL-Q1206-AppdxA-Tampadero Solar.pdf to data/1206/03_phase_2_study
Moved C8P2-NOL-Q1206-Attch1_Tampadero Solar.pdf to data/1206/03_phase_2_study
Moved C8P2-SCE-Northern-Q1208-Attch1-IF and Upgrades.pdf to data/1208/03_phase_2_study
Moved C8P2-SCE-Northern-Q1208-AppdxA-Antelope2Solar.pdf to data/1208/03_phase_2_study
Moved C8P2-SCE-Northern-Q1209-AppdxA - BradmoreES.pdf to data/1209/03_phase_2_study
Moved C8P2-SCE-Northern-Q1209-Attch1-IF and Upgrades.pdf to data/1209/03_phase_2_study
Moved C8P2-SCE-Eastern-Q1200-AppdxA-Victory Pass_rev1.pdf to data/1200/03_phase_2_study
Moved C8P2-SCE-Eastern-Q1200-AppdxA-Victory Pass.pdf to data/1200/03_phase_2_study
Moved C8P2-SCE-Eastern-Q1200-Attch1.pdf to data/1200/03_phase_2_study
Moved C8P2-NOL_Q1207-AppdxA-Yucca Solar Farm.pdf to data/1207/03_phase_2_study
Moved C8P2-NOL-Q1207-Attch1_Yucca Solar Farm.pdf to data/1207/03_phase_2_study
Moved QC13PII EOP Attch1 TOT967-Q1795 Arida Solar Farm 3.pdf to data/1795/03_phase_2_study
Moved QC13

In [371]:
import os
import shutil
import re

# Set the directory
new_directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/"
os.chdir(new_directory)

pdf_dir = "data"  # Replace with your actual directory

# Create regex pattern to match P2 in filenames
p2_pattern = re.compile(r'p2', re.IGNORECASE)

# Iterate through each project folder inside pdf_dir
for project_folder in os.listdir(pdf_dir):
    project_path = os.path.join(pdf_dir, project_folder)
    
    if os.path.isdir(project_path):
        phase_1_study_folder = os.path.join(project_path, '02_phase_1_study')
        phase_2_study_folder = os.path.join(project_path, '03_phase_2_study')

        # Ensure the Phase 2 study folder exists
        os.makedirs(phase_2_study_folder, exist_ok=True)

        # Check each PDF in the Phase 1 study folder
        if os.path.exists(phase_1_study_folder):
            for filename in os.listdir(phase_1_study_folder):
                file_path = os.path.join(phase_1_study_folder, filename)
                if filename.endswith(".pdf") and p2_pattern.search(filename):
                    # Move the PDF to the Phase 2 study folder
                    shutil.move(file_path, os.path.join(phase_2_study_folder, filename))
                    print(f"Moved {filename} to {phase_2_study_folder}")


Moved C9P2-EAST-Q1299-AppendixA.pdf to data/1299/03_phase_2_study
Moved C12P2-GLW-Q1654-Appendix A-GLW-Attachment 2-addendum.pdf to data/1654/03_phase_2_study
Moved C13P2-GLW-Q1801-Appendix A-GLW-Attachment 2.pdf to data/1801/03_phase_2_study
Moved C12P2-GLW-Q1655-Appendix A-GLW-Attachment 2-addendum.pdf to data/1655/03_phase_2_study
Moved C13P2-GLW-Q1800-Appendix A-GLW-Attachment 2.pdf to data/1800/03_phase_2_study
Moved C9P2-Northern-Q1326-Sagebrush Solar 1-AppendixA.pdf to data/1326/03_phase_2_study
Moved C9P2-Northern-Q1326-Sagebrush Solar 1-AppendixA-Attachment 2-Addendum.pdf to data/1326/03_phase_2_study
Moved C9P2-Northern-Q1326-Sagebrush Solar 1-AppendixA-Addendum.pdf to data/1326/03_phase_2_study
Moved C9P2-Northern-Q1326-Sagebrush Solar 1-AppendixA-Attachment 1-Addendum.pdf to data/1326/03_phase_2_study
Moved C9P2-Northern-Q1326-Sagebrush Solar 1-AppendixA-Attachment 2.pdf to data/1326/03_phase_2_study
Moved C9P2-Northern-Q1326-Sagebrush Solar 1-AppendixA-Attachment 1.pdf to 

In [378]:
import os
import shutil
import re

# Set the directory
new_directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/"
os.chdir(new_directory)

pdf_dir = "data"  # Replace with your actual directory

# Create regex pattern to match filenames containing "P1"
p1_pattern = re.compile(r'pI', re.IGNORECASE)

# Iterate through each project folder inside pdf_dir
for project_folder in os.listdir(pdf_dir):
    project_path = os.path.join(pdf_dir, project_folder)
    
    if os.path.isdir(project_path):
        phase_1_study_folder = os.path.join(project_path, '02_phase_1_study')
        phase_2_study_folder = os.path.join(project_path, '03_phase_2_study')

        # Ensure the Phase 1 study folder exists
        os.makedirs(phase_1_study_folder, exist_ok=True)

        # Check each PDF in the Phase 2 study folder
        if os.path.exists(phase_2_study_folder):
            for filename in os.listdir(phase_2_study_folder):
                file_path = os.path.join(phase_2_study_folder, filename)
                if filename.endswith(".pdf") and p1_pattern.search(filename):
                    # Move the PDF to the Phase 1 study folder
                    shutil.move(file_path, os.path.join(phase_1_study_folder, filename))
                    print(f"Moved {filename} to {phase_1_study_folder}")
 




Moved QC7PII-SCE-EOP-Q1069-Apendix A-BoulderSolar-Atchmnt1.pdf to data/1069/02_phase_1_study
Moved QC7PII-SCE-EOP-Q1069-Appendix A-BoulderSolar.pdf to data/1069/02_phase_1_study
Moved C7PII-SCE-EOP-Q1069-Appendix A-BoulderSolar-Attachment 2a.pdf to data/1069/02_phase_1_study
Moved C7PII-SCE-EOP-Q1069-Appendix A-BoulderSolar-Attachment 2.pdf to data/1069/02_phase_1_study
Moved QC14PII-SCE-Eastern-Q2049-TOT1123-Ardilla-Appendix A.pdf to data/2049/02_phase_1_study
Moved QC14PII-SCE-Eastern-Q2049-TOT1123-Ardilla-Appendix A_rev1.pdf to data/2049/02_phase_1_study
Moved QC14PII-SCE-Eastern-Q2049-TOT1123-Ardilla-Appendix A-Attachment 2.pdf to data/2049/02_phase_1_study
Moved QC14PII-SCE-Eastern-Q2049-TOT1123-Ardilla-Appendix A-Attachment 1.pdf to data/2049/02_phase_1_study
Moved QC14PII-SCE-Eastern-Q2049-TOT1123-Ardilla-Appendix A_Revision.pdf to data/2049/02_phase_1_study
Moved QC14PII-SCE-Eastern-Q2049-TOT1123-Ardilla-Appendix A-Attachment 2_Revision.pdf to data/2049/02_phase_1_study
Moved Q

In [391]:
import os
import shutil
import re

# Set the directory
new_directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/"
os.chdir(new_directory)

pdf_dir = "data"  # Replace with your actual directory

# Create a regex pattern to match filenames containing "capability"
capability_pattern = re.compile(r'megapack', re.IGNORECASE)

# Iterate through each project folder inside pdf_dir
for project_folder in os.listdir(pdf_dir):
    # Only process projects after 1891
    if project_folder.isdigit() and int(project_folder) > 1891:
        project_path = os.path.join(pdf_dir, project_folder)

        if os.path.isdir(project_path):
            phase_1_study_folder = os.path.join(project_path, '02_phase_1_study')
            document_dump_folder = os.path.join(project_path, 'document_dump')

            # Ensure the document dump folder exists
            os.makedirs(document_dump_folder, exist_ok=True)

            # Check each file in the Phase 1 study folder
            if os.path.exists(phase_1_study_folder):
                for filename in os.listdir(phase_1_study_folder):
                    file_path = os.path.join(phase_1_study_folder, filename)
                    if filename.endswith(".pdf") and capability_pattern.search(filename):
                        # Move the file to the document dump folder
                        shutil.move(file_path, os.path.join(document_dump_folder, filename))
                        print(f"Moved {filename} to {document_dump_folder}")



 
 



Moved Megapack_Short_Circuit_Aspen_OneLiner_Application_Note.pdf to data/1919/document_dump
Moved Manufacture_Datasheet_Megapack.pdf to data/1919/document_dump
Moved Megapack_Short_Circuit_Aspen_OneLiner_Application_Note.pdf to data/1910/document_dump
Moved Manufacture_Datasheet_Megapack.pdf to data/1910/document_dump
Moved Megapack_Datasheet_NG2_12.pdf to data/1973/document_dump
Moved Megapack_datasheet.pdf to data/1973/document_dump
Moved Megapack_Specification_v12_NG2_072621.pdf to data/1973/document_dump
Moved Megapack_ShortCircuit_Current_Letter.pdf to data/1973/document_dump
Moved Megapack_Specification_v12_NG2.pdf to data/1973/document_dump
Moved Megapack_Interconnection_Data.pdf to data/1973/document_dump
Moved Megapack_Specs.pdf to data/2135/document_dump
Moved Megapack_Interconnection_Data.pdf to data/2135/document_dump
Moved Megapack_Short_Circuit_Aspen_OneLiner_Application_Note1.pdf to data/1911/document_dump
Moved Megapack_Short_Circuit_Aspen_OneLiner_Application_Note.pdf 

In [395]:
import os
import shutil
import re

# Set the directory
new_directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/"
os.chdir(new_directory)

pdf_dir = "data"  # Replace with your actual directory

# Create regex patterns
attachment1_pattern = re.compile(r'attachment[\s_-]*1', re.IGNORECASE)
pii_attachment1_pattern = re.compile(r'(pii|p2).*attachment[\s_-]*1', re.IGNORECASE)  # Allowing anything between PII/P2 and Attachment 1


# Iterate through each project folder inside pdf_dir
for project_folder in os.listdir(pdf_dir):
    # Only process projects from 2064 onwards
    if project_folder.isdigit() and int(project_folder) >= 2064:
        project_path = os.path.join(pdf_dir, project_folder)

        if os.path.isdir(project_path):
            phase_1_study_folder = os.path.join(project_path, '02_phase_1_study')
            phase_2_study_folder = os.path.join(project_path, '03_phase_2_study')

            # Ensure the Phase 1 and Phase 2 study folders exist
            os.makedirs(phase_1_study_folder, exist_ok=True)
            os.makedirs(phase_2_study_folder, exist_ok=True)

            # Walk through all subdirectories and files in the project folder
            for root, dirs, files in os.walk(project_path):
                for filename in files:
                    file_path = os.path.join(root, filename)
                    if filename.endswith(".pdf"):
                        # Move PII Attachment 1 or P2 Attachment 1 to Phase 2 folder
                        if pii_attachment1_pattern.search(filename):
                            shutil.move(file_path, os.path.join(phase_2_study_folder, filename))
                            print(f"Moved {filename} to {phase_2_study_folder}")

                        # Move any other Attachment 1 to Phase 1 folder
                        elif attachment1_pattern.search(filename):
                            shutil.move(file_path, os.path.join(phase_1_study_folder, filename))
                            print(f"Moved {filename} to {phase_1_study_folder}")



Moved Attachment1- Northern TOT1120 Q2085.pdf to data/2085/02_phase_1_study
Moved QC14PII-Northern-Attachment1-TOT1120 Q2085 Solsken-clean.pdf to data/2085/03_phase_2_study
Moved Attachment1- Northern TOT1074 Q2071.pdf to data/2071/02_phase_1_study
Moved Attachment1- Northern TOT1097 Q2078.pdf to data/2078/02_phase_1_study
Moved QC14PII-Northern-Attachment1-TOT1097 Q2078 Soleil Cantil-clean.pdf to data/2078/03_phase_2_study
Moved Attachment1- Northern TOT1110 Q2079.pdf to data/2079/02_phase_1_study
Moved Attachment1- Northern TOT1112 Q2083.pdf to data/2083/02_phase_1_study
Moved Attachment1- Northern TOT1075 Q2070.pdf to data/2070/02_phase_1_study
Moved Attachment1- Northern TOT1119 Q2084.pdf to data/2084/02_phase_1_study
Moved Q2151-TOT1152-Attachment 1-SCE AS.pdf to data/2151/02_phase_1_study
Moved QC14P1-DesertLink-Q2151-Appendix A-Attachment 1.pdf to data/2151/02_phase_1_study
Moved QC14PII-SCE-ApndixA-Attachment 1  Q2105 - TOT1125 NOL.pdf to data/2105/03_phase_2_study
Moved QC14PI

In [398]:
import os
import shutil
import re

# Set the directory
new_directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/"
os.chdir(new_directory)

pdf_dir = "data"  # Replace with your actual directory

# Create regex pattern to match "Ph2" in filenames
ph2_pattern = re.compile(r'ph2', re.IGNORECASE)

# Iterate through each project folder inside pdf_dir
for project_folder in os.listdir(pdf_dir):
    # Only process projects from 2105 onwards
    if project_folder.isdigit() and int(project_folder) >= 2105:
        project_path = os.path.join(pdf_dir, project_folder)

        if os.path.isdir(project_path):
            phase_1_study_folder = os.path.join(project_path, '02_phase_1_study')
            phase_2_study_folder = os.path.join(project_path, '03_phase_2_study')

            # Ensure the Phase 2 study folder exists
            os.makedirs(phase_2_study_folder, exist_ok=True)

            # Check files under the Phase 1 folder
            if os.path.exists(phase_1_study_folder):
                for filename in os.listdir(phase_1_study_folder):
                    file_path = os.path.join(phase_1_study_folder, filename)
                    if filename.endswith(".pdf") and ph2_pattern.search(filename):
                        # Move the file to the Phase 2 study folder
                        shutil.move(file_path, os.path.join(phase_2_study_folder, filename))
                        print(f"Moved {filename} to {phase_2_study_folder}")
 

Moved QC14 Ph2 Attachment 1  TOT1090 Q2134 M.pdf to data/2134/03_phase_2_study
Moved QC14 Ph2 Attachment 1  TOT1060 Q2121 M.pdf to data/2121/03_phase_2_study
Moved QC14 Ph2 Attachment 1  TOT1034 Q2117 M.pdf to data/2117/03_phase_2_study
Moved QC14 Ph2 Attachment 1  TOT1148 Q2129 M.pdf to data/2129/03_phase_2_study
Moved QC14 Ph2 Attachment 1  TOT1033 Q2116 M.pdf to data/2116/03_phase_2_study
Moved QC14 Ph2 Attachment 1  TOT1033 Q2116 M rev1.pdf to data/2116/03_phase_2_study
Moved QC14 Ph2 Attachment 1  TOT1076 Q2127 M.pdf to data/2127/03_phase_2_study
Moved QC14 Ph2 Attachment 1  TOT1088 Q2131 M.pdf to data/2131/03_phase_2_study
Moved QC14 Ph2 Attachment 1  TOT1108 Q2136 M.pdf to data/2136/03_phase_2_study
Moved QC14 Ph2 Attachment 1  TOT1133 Q2138 M.pdf to data/2138/03_phase_2_study
Moved QC14 Ph2 Attachment 1  TOT1063 Q2125 M.pdf to data/2125/03_phase_2_study
Moved QC14 Ph2 Attachment 1  TOT1028 Q2113 M.pdf to data/2113/03_phase_2_study
Moved QC14 Ph2 Attachment 1  TOT1062 Q2124 M.pd

In [400]:
import os
import shutil
import re

# Set the directory
new_directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/"
os.chdir(new_directory)

pdf_dir = "data"  # Replace with your actual directory

# Create the base filename patterns for phase 1 and phase 2 files (using 'QXXXX' as a placeholder for project number)
phase_1_file_patterns = [
    r"QXXXX-TOT1085-Appendix A-SCE AS\.pdf",
    r"QXXXX-TOT1085-Attachment 1-SCE AS\.pdf",
    r"QXXXX-TOT1085-Attachment 2-SCE AS\.pdf",
    r"QXXXX-TOT1085-Attachment 3-SCE AS\.pdf",
    r"QC14P1-QXXXX_GLW-ApndxA\.pdf",
    r"QC14P1-QXXXX_GLW-Att1\.pdf",
    r"QC14P1-QXXXX_GLW-Att2\.pdf",
    
]

phase_2_file_patterns = [
    r"QC14 Ph2 Attachment 2 TOT1085 QXXXX EOP AFS\.pdf",
    r"QC14 Ph2 Attachment 3 TOT1085 QXXXX EOP AFS\.pdf",
    r"C14\.2-AS-EOP-QXXXX-TOT1085-WaterRock2_ApndxA\.pdf",
    r"QXXXXcombinedAttachment3Feb162024\.pdf",
    r"QC14P2-QXXXX_ApndxA-final\.pdf",
    r"QC14P2-QXXXX_Att1-final\.pdf",
    r"QC14P2-QXXXX_Att2_Update_2_16\.pdf",
    r"QC14P2-QXXXX_Att2-final\.pdf"
]

# Function to replace "QXXXX" with the actual project number
def replace_project_number(patterns, project_number):
    return [re.sub(r"QXXXX", f"Q{project_number}", pattern) for pattern in patterns]

# Iterate through each project folder inside pdf_dir
for project_folder in os.listdir(pdf_dir):
    # Only process projects from 2146 onwards
    if project_folder.isdigit() and int(project_folder) >= 2146:
        project_path = os.path.join(pdf_dir, project_folder)
        project_number = project_folder  # Use the project folder name as the project number (e.g., 2146)

        if os.path.isdir(project_path):
            phase_1_study_folder = os.path.join(project_path, '02_phase_1_study')
            phase_2_study_folder = os.path.join(project_path, '03_phase_2_study')

            # Ensure the Phase 1 and Phase 2 study folders exist
            os.makedirs(phase_1_study_folder, exist_ok=True)
            os.makedirs(phase_2_study_folder, exist_ok=True)

            # Update the filename patterns with the actual project number
            phase_1_file_patterns_project = replace_project_number(phase_1_file_patterns, project_number)
            phase_2_file_patterns_project = replace_project_number(phase_2_file_patterns, project_number)

            # Walk through all files in the project folder
            for root, dirs, files in os.walk(project_path):
                for filename in files:
                    file_path = os.path.join(root, filename)
                    if filename.endswith(".pdf"):
                        # Check for Phase 1 file patterns
                        if any(re.match(pattern, filename) for pattern in phase_1_file_patterns_project):
                            shutil.move(file_path, os.path.join(phase_1_study_folder, filename))
                            print(f"Moved {filename} to {phase_1_study_folder}")

                        # Check for Phase 2 file patterns
                        elif any(re.match(pattern, filename) for pattern in phase_2_file_patterns_project):
                            shutil.move(file_path, os.path.join(phase_2_study_folder, filename))
                            print(f"Moved {filename} to {phase_2_study_folder}")


Moved Q2150combinedAttachment3Feb162024.pdf to data/2150/03_phase_2_study
Moved QC14P2-Q2150_ApndxA-final.pdf to data/2150/03_phase_2_study
Moved QC14P2-Q2150_Att1-final.pdf to data/2150/03_phase_2_study
Moved QC14P2-Q2150_Att2-final.pdf to data/2150/03_phase_2_study
Moved QC14P2-Q2150_Att2_Update_2_16.pdf to data/2150/03_phase_2_study
Moved QC14P2-Q2147_Att1-final.pdf to data/2147/03_phase_2_study
Moved QC14P2-Q2147_ApndxA-final.pdf to data/2147/03_phase_2_study
Moved QC14P1-Q2147_GLW-ApndxA.pdf to data/2147/02_phase_1_study
Moved Q2149combinedAttachment3Feb162024.pdf to data/2149/03_phase_2_study
Moved QC14P2-Q2149_Att2_Update_2_16.pdf to data/2149/03_phase_2_study
Moved QC14P2-Q2149_ApndxA-final.pdf to data/2149/03_phase_2_study
Moved QC14P1-Q2149_GLW-ApndxA.pdf to data/2149/02_phase_1_study
Moved QC14P2-Q2148_ApndxA-final.pdf to data/2148/03_phase_2_study
Moved Q2148combinedAttachment3Feb162024.pdf to data/2148/03_phase_2_study
Moved QC14P2-Q2148_Att2_Update_2_16.pdf to data/2148/0

In [402]:
import os
import csv

# Set the directory
new_directory = "/Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/"
os.chdir(new_directory)

pdf_dir = "data"  # Replace with your actual directory

# CSV output file
output_csv = "project_phase_status.csv"

# Prepare the CSV header
header = ['q_id', 'phase 1', 'phase 2', 'reassessment', 'multiple phase 1', 'multiple phase 2']

# Function to check if a folder contains PDF files and return the count
def check_folder(folder_path):
    if not os.path.exists(folder_path):
        return "no", 0
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
    return ("yes" if pdf_files else "no"), len(pdf_files)

# Open the CSV file for writing
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)  # Write the header row

    # Iterate through each project folder inside pdf_dir
    for project_folder in os.listdir(pdf_dir):
        project_path = os.path.join(pdf_dir, project_folder)

        if os.path.isdir(project_path):
            q_id = project_folder  # Use project folder name as q_id

            # Define the folder paths for phase 1, phase 2, and reassessment
            phase_1_folder = os.path.join(project_path, '02_phase_1_study')
            phase_2_folder = os.path.join(project_path, '03_phase_2_study')
            reassessment_folder = os.path.join(project_path, '05_reassesment') 

            # Check the folders and get the status and number of PDFs
            phase_1_status, phase_1_count = check_folder(phase_1_folder)
            phase_2_status, phase_2_count = check_folder(phase_2_folder)
            reassessment_status, _ = check_folder(reassessment_folder)

            # Write the data row to the CSV file
            writer.writerow([q_id, phase_1_status, phase_2_status, reassessment_status, phase_1_count, phase_2_count])

print(f"CSV file '{output_csv}' has been created successfully.")


CSV file 'project_phase_status.csv' has been created successfully.
